In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model
from keras import backend as K

import tensorflow as tf

import os
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import aparent.visualization as vis

from aparent.predictor import *

import urllib
import urllib.request
import pickle
from time import sleep


Using TensorFlow backend.


In [2]:
#Load APADB Data and filter on targeted genes

#genes = ['RUNX1', 'CEBPA', 'GATA2', 'ANKRD26', 'DDX41', 'ETV6', 'PTEN', 'BRCA1', 'BRCA2', 'TP53', 'APC', 'ATM', 'PALB2', 'MSH2', 'MLH1', 'MSH6', 'PMS2', 'MUTYH']

polyadb_df = pd.read_csv('../../../data/native_data/polyadb/processed_data/polyadb_processed.csv', sep=',')

#polyadb_df = polyadb_df.loc[polyadb_df['gene'].isin(genes)].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[~polyadb_df['gene'].isna()].reset_index(drop=True).copy()
polyadb_df = polyadb_df.loc[polyadb_df['gene'] != 'na'].reset_index(drop=True).copy()

print('len(polyadb_df) = ' + str(len(polyadb_df)))


len(polyadb_df) = 190117


In [3]:
polyadb_df_minus = polyadb_df.query("strand == '-'").copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df.query("strand == '+'").copy().reset_index(drop=True)

polyadb_df_minus = polyadb_df_minus.sort_values(by='pas_pos', ascending=False).copy().reset_index(drop=True)
polyadb_df_plus = polyadb_df_plus.sort_values(by='pas_pos', ascending=True).copy().reset_index(drop=True)

new_gene_id_list_plus = []
sitenum_list_plus = []
gene_id_dict = {}
for _, row in polyadb_df_plus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_plus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_plus.append(gene_id_dict[gene])

polyadb_df_plus['gene_id'] = new_gene_id_list_plus
polyadb_df_plus['sitenum'] = sitenum_list_plus

new_gene_id_list_minus = []
sitenum_list_minus = []
gene_id_dict = {}
for _, row in polyadb_df_minus.iterrows() :

    gene = str(row['gene'])

    if gene not in gene_id_dict :
        gene_id_dict[gene] = 0

    gene_id_dict[gene] += 1

    new_gene_id_list_minus.append(gene + "." + str(gene_id_dict[gene]))
    sitenum_list_minus.append(gene_id_dict[gene])

polyadb_df_minus['gene_id'] = new_gene_id_list_minus
polyadb_df_minus['sitenum'] = sitenum_list_minus

polyadb_df = pd.concat([polyadb_df_plus, polyadb_df_minus])

polyadb_df = polyadb_df.sort_values(by=['gene', 'sitenum'], ascending=True).reset_index(drop=True).copy()

In [4]:
polyadb_df.head()

,gene,gene_id,sitenum,num_sites,pas,seq,seq_ext,wide_seq,wide_seq_ext,rpm,total_rpm,ratio,pas_pos,cut_mode,chrom,strand,site_type
0,A1BG,A1BG.1,1,11,0,TTCTGCTAGGCTGGGCTAGTATCTTCCATGGCAAGATACTCAAACT...,TAGGGAATATAACTTTTAAAATGCGAAAGTATATTATACATATTGC...,AGTTTGGCTAGCATTTCAAAATGGATAGGGAATATAACTTTTAAAA...,GCTTTCAAAATTCCTGATTAGCTTACAATTAATTGCAAATAACTTC...,1.564234,24.390222,0.064134,58859852,58859830,chr19,-,Intron
1,A1BG,A1BG.2,2,11,-1,AGGTGCCGCTACCGCTCCTGGGTGCCCCACACCTTCGAATCGGAGC...,CGGCGAACCTCGAGCTGATCTTCGTGGGGCCCCAGCACGCCGGCAA...,AAGACGGTCCGCACCCCCGGGGCCGCGGCGAACCTCGAGCTGATCT...,TCCCCGACGTCACCTTCGAGCTGCTGCGCGAGGGCGAGACGAAGGC...,1.740036,24.390222,0.071342,58858744,58858719,chr19,-,Internal_exon
2,A1BG,A1BG.3,3,11,0,CCGGGGTCTCTGGGGGGCTGGAGAAGCCTCCCTCATTCCTCCCAGG...,AGTGCCGGGGATTCTGGACTGGCTCCCTCCCCTCCTGTTGCAGCAC...,CCAGGGTGCTGTTGGTGTCCTCAGAAGTGCCGGGGATTCTGGACTG...,CACCATTTCTCTTTTCTCCCGTTGTCTAGAAAGCTGATGCAGCCGC...,11.529986,24.390222,0.472730,58858249,58858220,chr19,-,3_most_exon
3,A1BG,A1BG.4,4,11,4,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,ACCACTGCCGAGTTTCTGCCTGGAGCAGGGCCTCAAGGCCCTGGCA...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,GATTCCTCTGTGTTCTTTCCCATTATCCCTCAGCTCTGAGTCCTCT...,1.386686,24.390222,0.056854,58857215,58857210,chr19,-,3_most_exon
4,A1BG,A1BG.5,5,11,4,AACGAATTTGCTAGTGATCTGGTTTACTGCCTTAGTAATATCTAGT...,GAAGATGCGTATCAGTATTATCAACCAATAGTTGATGAATTGTGTT...,CAGGGCCTCAAGGCCCTGGCACACAGAAGATGCGTATCAGTATTAT...,CCATCTGACATCTGATGCCTTCCCAACCACTGCCGAGTTTCTGCCT...,1.174863,24.390222,0.048169,58857165,58857142,chr19,-,3_most_exon


In [5]:

polyadb_df['start'] = polyadb_df['pas_pos'] - 150
polyadb_df['end'] = polyadb_df['pas_pos'] + 150

polyadb_bed = polyadb_df[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand']].copy().reset_index(drop=True)

polyadb_bed.head()

polyadb_bed.to_csv("polyadb_coordinates.bed", sep='\t', header=False, index=False)


In [6]:
#Tissue types to compile data for

tissue_types = [
    'Adipose_Subcutaneous',
    'Adipose_Visceral_Omentum',
    'Adrenal_Gland',
    'Artery_Aorta',
    'Artery_Coronary',
    'Artery_Tibial',
    'Brain_Amygdala',
    'Brain_Anterior_cingulate_cortex_BA24',
    'Brain_Caudate_basal_ganglia',
    'Brain_Cerebellar_Hemisphere',
    'Brain_Cerebellum',
    'Brain_Cortex',
    'Brain_Frontal_Cortex_BA9',
    'Brain_Hippocampus',
    'Brain_Hypothalamus',
    'Brain_Nucleus_accumbens_basal_ganglia',
    'Brain_Putamen_basal_ganglia',
    'Brain_Spinal_cord_cervical_c-1',
    'Brain_Substantia_nigra',
    'Breast_Mammary_Tissue',
    'Cells_EBV-transformed_lymphocytes',
    'Cells_Cultured_fibroblasts',
    'Colon_Sigmoid',
    'Colon_Transverse',
    'Esophagus_Gastroesophageal_Junction',
    'Esophagus_Mucosa',
    'Esophagus_Muscularis',
    'Heart_Atrial_Appendage',
    'Heart_Left_Ventricle',
    'Kidney_Cortex',
    'Liver',
    'Lung',
    'Minor_Salivary_Gland',
    'Muscle_Skeletal',
    'Nerve_Tibial',
    'Ovary',
    'Pancreas',
    'Pituitary',
    'Prostate',
    'Skin_Not_Sun_Exposed_Suprapubic',
    'Skin_Sun_Exposed_Lower_leg',
    'Small_Intestine_Terminal_Ileum',
    'Spleen',
    'Stomach',
    'Testis',
    'Thyroid',
    'Uterus',
    'Vagina',
    'Whole_Blood'
]


In [12]:
#Convert to hg19 coordinates (for compatability with polyadb)

batch_size = 500

rs_dict = {}

for tissue_type in tissue_types :
    
    print("Processing " + tissue_type + "...")
    
    tissue_df = pd.read_csv(tissue_type + ".3aQTL.txt", sep='\t')
    
    rs_ids_w_prefix = tissue_df['rs'].values.tolist()
    rs_ids = np.unique([rs[2:] for rs in rs_ids_w_prefix if rs not in rs_dict]).tolist()
    
    n_batches = len(rs_ids) // batch_size + (1 if len(rs_ids) % batch_size > 0 else 0)

    print("- n_batches = " + str(n_batches))
    
    for batch_ix in range(0, n_batches) :

        print("Processing rs record batch " + str(batch_ix) + "...")

        rs_id_batch = rs_ids[batch_ix * batch_size: (batch_ix + 1) * batch_size]
        rs_id_str = ",".join(rs_id_batch)

        !epost -db snp -id {rs_id_str} | efetch -format docsum | xtract -pattern DocumentSummary -element SNP_ID CHRPOS_PREV_ASSM ALLELE SNP_CLASS > eutils_temp1.txt

        with open("eutils_temp1.txt", "rt") as f :

            for line_raw in f.readlines() :
                line = line_raw.strip()

                lineparts = line.split("\t")

                if len(lineparts) == 4 :
                    rs_id, chrom_pos, alt, snp_type = lineparts

                    rs_dict["rs" + str(rs_id)] = [chrom_pos.split(":")[0], chrom_pos.split(":")[1], alt, snp_type]

                else :
                    print("[Warning] Error processing one of the records.")
    
    chroms = []
    poses = []

    for _, row in tissue_df.iterrows() :

        rs_id = row['rs']

        if rs_id in rs_dict :
            chroms.append("chr" + str(rs_dict[rs_id][0]))
            poses.append(int(rs_dict[rs_id][1]))
        else :
            chroms.append("N/A")
            poses.append("N/A")

    tissue_df['chrom'] = chroms
    tissue_df['pos'] = poses
    tissue_df['ref'] = tissue_df['SNP'].apply(lambda x: x.split("_")[-3])
    tissue_df['alt'] = tissue_df['SNP'].apply(lambda x: x.split("_")[-2])
    
    tissue_df.to_csv(tissue_type + ".3aQTL_hg19.csv", sep='\t')

print("Done!")


Processing Adipose_Subcutaneous...
- n_batches = 780
Processing rs record batch 0...
Processing rs record batch 1...
Processing rs record batch 2...
Processing rs record batch 3...
Processing rs record batch 4...
Processing rs record batch 5...
Processing rs record batch 6...
Processing rs record batch 7...
Processing rs record batch 8...
Processing rs record batch 9...
Processing rs record batch 10...
Processing rs record batch 11...
Processing rs record batch 12...
Processing rs record batch 13...
[Warning] Error processing one of the records.
Processing rs record batch 14...
Processing rs record batch 15...
Processing rs record batch 16...
Processing rs record batch 17...
Processing rs record batch 18...
Processing rs record batch 19...
Processing rs record batch 20...
Processing rs record batch 21...
Processing rs record batch 22...
Processing rs record batch 23...
Processing rs record batch 24...
Processing rs record batch 25...
Processing rs record batch 26...
Processing rs recor

Processing rs record batch 218...
Processing rs record batch 219...
Processing rs record batch 220...
Processing rs record batch 221...
Processing rs record batch 222...
Processing rs record batch 223...
Processing rs record batch 224...
Processing rs record batch 225...
Processing rs record batch 226...
Processing rs record batch 227...
Processing rs record batch 228...
Processing rs record batch 229...
Processing rs record batch 230...
Processing rs record batch 231...
Processing rs record batch 232...
Processing rs record batch 233...
Processing rs record batch 234...
Processing rs record batch 235...
Processing rs record batch 236...
Processing rs record batch 237...
Processing rs record batch 238...
Processing rs record batch 239...
Processing rs record batch 240...
Processing rs record batch 241...
Processing rs record batch 242...
Processing rs record batch 243...
Processing rs record batch 244...
Processing rs record batch 245...
Processing rs record batch 246...
Processing rs 

Processing rs record batch 362...
Processing rs record batch 363...
[Warning] Error processing one of the records.
Processing rs record batch 364...
Processing rs record batch 365...
Processing rs record batch 366...
Processing rs record batch 367...
Processing rs record batch 368...
Processing rs record batch 369...
Processing rs record batch 370...
Processing rs record batch 371...
Processing rs record batch 372...
Processing rs record batch 373...
[Warning] Error processing one of the records.
Processing rs record batch 374...
Processing rs record batch 375...
Processing rs record batch 376...
Processing rs record batch 377...
Processing rs record batch 378...
Processing rs record batch 379...
Processing rs record batch 380...
Processing rs record batch 381...
Processing rs record batch 382...
Processing rs record batch 383...
[Warning] Error processing one of the records.
Processing rs record batch 384...
Processing rs record batch 385...
[Warning] Error processing one of the recor

[Warning] Error processing one of the records.
Processing rs record batch 523...
Processing rs record batch 524...
Processing rs record batch 525...
Processing rs record batch 526...
Processing rs record batch 527...
Processing rs record batch 528...
Processing rs record batch 529...
Processing rs record batch 530...
Processing rs record batch 531...
Processing rs record batch 532...
Processing rs record batch 533...
Processing rs record batch 534...
Processing rs record batch 535...
Processing rs record batch 536...
Processing rs record batch 537...
[Warning] Error processing one of the records.
Processing rs record batch 538...
[Warning] Error processing one of the records.
Processing rs record batch 539...
Processing rs record batch 540...
Processing rs record batch 541...
Processing rs record batch 542...
Processing rs record batch 543...
Processing rs record batch 544...
Processing rs record batch 545...
Processing rs record batch 546...
Processing rs record batch 547...
Processin

Processing rs record batch 665...
Processing rs record batch 666...
Processing rs record batch 667...
Processing rs record batch 668...
[Warning] Error processing one of the records.
Processing rs record batch 669...
Processing rs record batch 670...
Processing rs record batch 671...
Processing rs record batch 672...
Processing rs record batch 673...
Processing rs record batch 674...
Processing rs record batch 675...
Processing rs record batch 676...
Processing rs record batch 677...
Processing rs record batch 678...
Processing rs record batch 679...
Processing rs record batch 680...
Processing rs record batch 681...
Processing rs record batch 682...
Processing rs record batch 683...
Processing rs record batch 684...
Processing rs record batch 685...
Processing rs record batch 686...
Processing rs record batch 687...
Processing rs record batch 688...
Processing rs record batch 689...
[Warning] Error processing one of the records.
[Warning] Error processing one of the records.
[Warning]

...

Processing rs record batch 4...
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:04:22 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863686 [RS] OR rs281863676 [RS] OR rs281863661 [RS] OR rs281863522 [RS] OR rs281863401 [RS] OR rs281863349 [RS] OR rs281875005 [RS] OR rs281874939 [RS] OR rs281874818 [RS] OR rs281863219 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863676[RS]</PhraseNotFound>
  <PhraseNotFound>rs281875005[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863219[RS]</PhraseNotFound>
<ErrorList>>78
SECOND ATTEMPT>78
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:04:25 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863686 [RS] OR rs281863676 [RS] OR rs281863661 [RS] OR rs281863522 [RS] OR rs281863401 [RS] OR rs281863349 [RS] OR rs2818

nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863466 [RS] OR rs281863364 [RS] OR rs281863567 [RS] OR rs281863521 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult>
<eSearchResult>
  <Count>1</Count>
  <RetMax>0</RetMax>
  <RetStart>0</RetStart>
  <QueryKey>1</QueryKey>
  <WebEnv>MCID_6149d87ea2a33d50d1733e86</WebEnv>
  <TranslationStack>
    <TermSet>
      <Term>rs281863364[RS]</Term>
      <Field>RS</Field>
      <Count>1</Count>
      <Explode>N</Explode>
    </TermSet>
    <OP>GROUP</OP>
  </TranslationStack>
  <QueryTranslation>rs281863364[RS]</QueryTranslation>
  <ErrorList>
    <PhraseNotFound>rs281863466[RS]</PhraseNotFound>
    <PhraseNotFound>rs281863567[RS]</PhraseNotFound>
    <PhraseNotFound>rs281863521[RS]</PhraseNotFound>
  </ErrorList>
</eSearchResult>>78
QUERY FAILURE>78
[Warning] Error processing 

>78 WARNING: >78 FAILURE ( Tue Sep 21 06:05:41 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863724 [RS] OR rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863724[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863499[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863464[RS]</PhraseNotFound>
<ErrorList>>78
LAST ATTEMPT>78
>78 ERROR: >78 FAILURE ( Tue Sep 21 06:05:43 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863724 [RS] OR rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eSearchResult>
<eSearchResult>
  <Count>

Processing rs record batch 2...
[Warning] Error processing one of the records.
[Warning] Error processing one of the records.
Processing rs record batch 3...
Processing rs record batch 4...
[Warning] Error processing one of the records.
Processing rs record batch 5...
[Warning] Error processing one of the records.
[Warning] Error processing one of the records.
Processing rs record batch 6...
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:07:13 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863592 [RS] OR rs281863427 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863592[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863427[RS]</PhraseNotFound>
<ErrorList>>78
SECOND ATTEMPT>78
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:07:15 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -us

>78 WARNING: >78 FAILURE ( Tue Sep 21 06:12:28 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863499[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863464[RS]</PhraseNotFound>
<ErrorList>>78
SECOND ATTEMPT>78
>78 WARNING: >78 FAILURE ( Tue Sep 21 06:12:30 PDT 2021 )>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db snp -term "rs281863504 [RS] OR rs281863499 [RS] OR rs281863464 [RS] OR rs9274253 [RS]" -tool edirect -edirect 15.3 -edirect_os Linux -email jlinder2@cseubuntu
>78<ErrorList>
  <PhraseNotFound>rs281863499[RS]</PhraseNotFound>
  <PhraseNotFound>rs281863464[RS]</PhraseNotFound>
<ErrorList>>78
LAST ATTEMPT>78
>78 ERROR: >78 FAILU

In [7]:

version_suffix = '_v8_w_effect_sizes'


In [8]:
#Compile apaQTL data from GTEx (v8)

def get_snp_pos(row) :
    if row['strand'] == '+' :
        return row['snp_pos'] - row['pas_pos'] + 175 - 1
    else :
        return row['pas_pos'] - row['snp_pos'] + 175

def get_snp_ref(row) :
    if row['strand'] == '+' :
        return row['snp_ref']
    else :
        if row['snp_ref'] == 'A' :
            return 'T'
        if row['snp_ref'] == 'C' :
            return 'G'
        if row['snp_ref'] == 'G' :
            return 'C'
        if row['snp_ref'] == 'T' :
            return 'A'

def get_snp_var(row) :
    if row['strand'] == '+' :
        return row['snp_var']
    else :
        if row['snp_var'] == 'A' :
            return 'T'
        if row['snp_var'] == 'C' :
            return 'G'
        if row['snp_var'] == 'G' :
            return 'C'
        if row['snp_var'] == 'T' :
            return 'A'

def get_var_seq(row) :
    ref_seq = row['wide_seq_ext']
    var_seq = ref_seq[:row['rel_snp_pos']] + row['rel_snp_var'] + ref_seq[row['rel_snp_pos']+1:]
    
    return var_seq

for tissue_type in tissue_types :
    
    print("Processing data for tissue = '" + str(tissue_type) + "'.")
    
    snp_df = pd.read_csv(tissue_type + '.3aQTL_hg19.csv', sep='\t')
    
    snp_df.loc[snp_df['chrom'].isnull(), 'chrom'] = 'Missing'
    snp_df.loc[snp_df['pos'].isnull(), 'pos'] = -1

    snp_df['start'] = snp_df['pos'].astype(int)
    snp_df['end'] = (snp_df['pos'] + 1).astype(int)
    snp_df['gene'] = snp_df['transcript'].apply(lambda x: x.split("@")[1])
    
    snp_df = snp_df.query("pos != -1 and chrom != 'Missing'").copy().reset_index(drop=True)
    
    #Join and collect effect sizes
    effect_size_df = pd.read_csv('with_effect_sizes/' + tissue_type + '.3aQTL.txt', sep='\t')[['SNP', 'gene', 'beta']]
    snp_df['row_id'] = snp_df['SNP'] + "_" + snp_df['transcript'].apply(lambda x: x.split("@")[0])
    effect_size_df['row_id'] = effect_size_df['SNP'] + "_" + effect_size_df['gene'].apply(lambda x: x.split("|")[0])
    
    snp_df = snp_df.join(effect_size_df[['row_id', 'beta']].set_index('row_id'), on='row_id', how='inner').copy().reset_index(drop=True)

    snp_df = snp_df[['chrom', 'start', 'end', 'gene', 'SNP', 'rs', 'transcript', 'p.value', 'beta']]
    
    #Filter on lead SNPs
    #snp_df = snp_df.sort_values(by='p.value', ascending=True).drop_duplicates(subset=['transcript'], keep='first').copy().reset_index(drop=True)
    
    snp_df.to_csv(tissue_type + version_suffix + ".3aQTL.coordinates.bed", sep='\t', header=False, index=False)

    !bedtools intersect -a polyadb_coordinates.bed -b {tissue_type + version_suffix}.3aQTL.coordinates.bed -wa -wb > polyadb_{tissue_type + version_suffix}_intersect.bed

    snp_bed_hg19 = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_intersect.bed", sep='\t', error_bad_lines=False, index_col=False, names=['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'rs_id', 'transcript_id', 'p_val', 'effect_size'])
    
    snp_bed_hg19 = snp_bed_hg19.query("gene == gene_2").copy().reset_index(drop=True)
    
    snp_bed_hg19 = snp_bed_hg19[['chrom', 'start', 'end', 'gene', 'gene_id', 'strand', 'chr_2', 'snp_pos', 'end_2', 'gene_2', 'snp_id', 'transcript_id', 'p_val', 'effect_size']]

    snp_bed_hg19['snp_ref'] = snp_bed_hg19['snp_id'].apply(lambda s: s.split("_")[2])
    snp_bed_hg19['snp_var'] = snp_bed_hg19['snp_id'].apply(lambda s: s.split("_")[3])
    
    polyadb_df_sel = polyadb_df[['wide_seq_ext', 'gene_id', 'pas', 'pas_pos', 'cut_mode', 'site_type', 'rpm', 'total_rpm', 'ratio']].copy().reset_index(drop=True)

    snp_df = snp_bed_hg19.join(polyadb_df_sel.set_index('gene_id'), on='gene_id').copy().reset_index(drop=True)
    
    snp_df['rel_snp_pos'] = snp_df.apply(get_snp_pos, axis=1)
    snp_df['rel_snp_ref'] = snp_df.apply(get_snp_ref, axis=1)
    snp_df['rel_snp_var'] = snp_df.apply(get_snp_var, axis=1)

    snp_df['wide_seq_ext_var'] = snp_df.apply(get_var_seq, axis=1)
    
    #Find transcript ends in hg38
    
    snp_df['transcript_id'] = snp_df['transcript_id'].apply(lambda x: x.split("@")[0])
    snp_df['snp_transcript_id'] = snp_df['snp_id'] + "_" + snp_df['transcript_id']
    
    transcript_end_dict = { row['transcript_id'] : -1 for _, row in snp_df.iterrows() }

    with open("/home/jlinder2/Downloads/GRCh38_latest_genomic.gtf", 'rt') as f :
        for line_raw in f.readlines() :

            a = line_raw.strip()

            if len(a.split("\t")) > 8 :
                start = a.strip().split("\t")[3]
                end = a.strip().split("\t")[4]

                strand = a.strip().split("\t")[6]

                t_id = ""
                if strand in ['+', '-'] :
                    t_id = a.strip().split("\t")[8].split(";")[1].split("\"")[1].split(".")[0]

                if t_id in transcript_end_dict :

                    if transcript_end_dict[t_id] == -1 :
                        if strand == '+' :
                            transcript_end_dict[t_id] = int(end)
                        else :
                            transcript_end_dict[t_id] = int(start)
                    else :
                        if strand == '+' :
                            transcript_end_dict[t_id] = max(transcript_end_dict[t_id], int(end))
                        else :
                            transcript_end_dict[t_id] = min(transcript_end_dict[t_id], int(start))

    qtl_cut_poses = []
    for _, row in snp_df.iterrows() :

        qtl_cut_poses.append(transcript_end_dict[row['transcript_id']])

    snp_df['snp_pos_hg38'] = snp_df['snp_id'].apply(lambda x: int(x.split("_")[1]))
    
    snp_df['qtl_cut_pos'] = qtl_cut_poses
    snp_df['snp_distance'] = np.abs(snp_df['qtl_cut_pos'] - snp_df['snp_pos_hg38'])
    
    #Calculate mean PDUIs
    
    transcript_ids = np.unique(snp_df['transcript_id'].apply(lambda x: x.split("@")[0]).values).tolist()
    transcript_df = pd.DataFrame({'transcript_id' : transcript_ids})
    
    pdui_df = None
    try :
        pdui_df = pd.read_csv(tissue_type + "_combined_All_PDUIs.txt", sep='\t')
    except FileNotFoundError :
        continue
    
    pdui_df['event_id'] = pdui_df['Gene'].apply(lambda x: x.split("|")[0])
    pdui_df = pdui_df.join(transcript_df[['transcript_id']].set_index('transcript_id'), on='event_id', how='inner').copy().reset_index(drop=True)

    gtex_samples = pdui_df.columns.values.tolist()[2:-1]

    def calc_mean_PDUI(row) :

        n_samples = 0.
        PDUI = 0.

        for gtex_sample in gtex_samples :
            
            if not np.isnan(row[gtex_sample]) :

                PDUI += row[gtex_sample]
                n_samples += 1.

        return PDUI / n_samples

    pdui_df['mean_PDUI'] = pdui_df.apply(calc_mean_PDUI, axis=1)
    
    median_mean_PDUI = np.median(pdui_df['mean_PDUI'].values)
    
    tissue_transcript_PDUI_dict = {}
    for _, row in pdui_df.iterrows() :
        tissue_transcript_PDUI_dict[row['event_id']] = row['mean_PDUI']

    PDUIs = []
    for _, row in snp_df.iterrows() :

        mean_PDUI = -1.
        if row['transcript_id'].split("@")[0] in tissue_transcript_PDUI_dict :
            mean_PDUI = tissue_transcript_PDUI_dict[row['transcript_id'].split("@")[0]]
        
        PDUIs.append(mean_PDUI)

    snp_df['mean_PDUI'] = PDUIs
    snp_df['snp_pos'] = snp_df['rel_snp_pos'] - 105
    
    snp_df = snp_df.query("mean_PDUI != -1.").copy().reset_index(drop=True)
    
    snp_df = snp_df.join(polyadb_df[['gene_id', 'sitenum', 'num_sites']].set_index("gene_id"), on='gene_id', how='inner').copy().reset_index(drop=True)
    snp_df['distal_snp'] = (snp_df['num_sites'] - snp_df['sitenum']).apply(lambda x: 1 if x == 0 else -1)
    
    #Drop duplicates
    snp_df['row_id'] = snp_df['gene_id'] + "_" + snp_df['snp_id'] + "_" + snp_df['transcript_id']
    snp_df = snp_df.sort_values(by='snp_id').drop_duplicates(subset=['row_id'], keep='first')

    print(" - len(snp_df) = " + str(len(snp_df)))
    
    #Save final apaQTL dataframe
    snp_df.to_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')


Processing data for tissue = 'Adipose_Subcutaneous'.
 - len(snp_df) = 11790
Processing data for tissue = 'Adipose_Visceral_Omentum'.
 - len(snp_df) = 8958
Processing data for tissue = 'Adrenal_Gland'.
 - len(snp_df) = 4617
Processing data for tissue = 'Artery_Aorta'.
 - len(snp_df) = 7743
Processing data for tissue = 'Artery_Coronary'.
 - len(snp_df) = 3329
Processing data for tissue = 'Artery_Tibial'.
 - len(snp_df) = 10664
Processing data for tissue = 'Brain_Amygdala'.
 - len(snp_df) = 1511
Processing data for tissue = 'Brain_Anterior_cingulate_cortex_BA24'.
 - len(snp_df) = 2037
Processing data for tissue = 'Brain_Caudate_basal_ganglia'.
 - len(snp_df) = 3063
Processing data for tissue = 'Brain_Cerebellar_Hemisphere'.
 - len(snp_df) = 2733
Processing data for tissue = 'Brain_Cerebellum'.
 - len(snp_df) = 3677
Processing data for tissue = 'Brain_Cortex'.
 - len(snp_df) = 3505
Processing data for tissue = 'Brain_Frontal_Cortex_BA9'.
 - len(snp_df) = 2614
Processing data for tissue = '

In [9]:
#Concatenate dataframes

snp_dfs = []

for tissue_type in tissue_types :
    
    print("Processing tissue = '" + tissue_type + "'")
    
    snp_df = None
    try :
        snp_df = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')
    except FileNotFoundError :
        continue
    
    snp_df['tissue_type'] = tissue_type
    
    snp_dfs.append(snp_df)

snp_df = pd.concat(snp_dfs).copy().reset_index(drop=True)


Processing tissue = 'Adipose_Subcutaneous'
Processing tissue = 'Adipose_Visceral_Omentum'
Processing tissue = 'Adrenal_Gland'
Processing tissue = 'Artery_Aorta'
Processing tissue = 'Artery_Coronary'
Processing tissue = 'Artery_Tibial'
Processing tissue = 'Brain_Amygdala'
Processing tissue = 'Brain_Anterior_cingulate_cortex_BA24'
Processing tissue = 'Brain_Caudate_basal_ganglia'
Processing tissue = 'Brain_Cerebellar_Hemisphere'
Processing tissue = 'Brain_Cerebellum'
Processing tissue = 'Brain_Cortex'
Processing tissue = 'Brain_Frontal_Cortex_BA9'
Processing tissue = 'Brain_Hippocampus'
Processing tissue = 'Brain_Hypothalamus'
Processing tissue = 'Brain_Nucleus_accumbens_basal_ganglia'
Processing tissue = 'Brain_Putamen_basal_ganglia'
Processing tissue = 'Brain_Spinal_cord_cervical_c-1'
Processing tissue = 'Brain_Substantia_nigra'
Processing tissue = 'Breast_Mammary_Tissue'
Processing tissue = 'Cells_EBV-transformed_lymphocytes'
Processing tissue = 'Cells_Cultured_fibroblasts'
Processing

In [10]:

merged_df = snp_df


In [11]:

merged_df.to_csv("polyadb_merged" + version_suffix + "_SNPs.csv", sep='\t')


In [12]:
#Compute median effect sizes

exclude_tissues = [['']]
exclude_suffixes = ['']

for exclude_tissue, exclude_suffix in zip(exclude_tissues, exclude_suffixes) :

    print("Excluding " + str(exclude_tissue) + "...")
    
    snp_dict = {}

    for i, tissue_type in enumerate(tissue_types) :
        
        if tissue_type in exclude_tissue :
            continue

        print("Processing tissue = '" + tissue_type + "'")

        snp_df = None
        try :
            snp_df = pd.read_csv("polyadb_" + tissue_type + version_suffix + "_SNPs.csv", sep='\t')
        except FileNotFoundError :
            continue

        for _, row in snp_df.iterrows() :

            snp_id = row['snp_id'] + "__" + row['gene_id'] + "__" + row['transcript_id']

            effect_size = row['effect_size']
            p_val = row['p_val']

            if snp_id not in snp_dict :
                snp_dict[snp_id] = {
                    'effect_size' : [],
                    'p_val' : [],
                    'snp_id' : row['snp_id'],
                    'gene' : row['gene'],
                    'gene_id' : row['gene_id'],
                    'pas' : row['pas'],
                    'wide_seq_ext' : row['wide_seq_ext'],
                    'wide_seq_ext_var' : row['wide_seq_ext_var'],
                    'rel_snp_pos' : row['rel_snp_pos'],
                    'snp_pos' : row['rel_snp_pos'] - 105,
                    'mean_PDUI' : row['mean_PDUI'],
                    'site_type' : row['site_type'],
                    'distal_snp' : row['distal_snp'],
                    'snp_distance' : row['snp_distance'],
                }

            snp_dict[snp_id]['effect_size'].append(effect_size)
            snp_dict[snp_id]['p_val'].append(p_val)

    extended_snp_ids = []
    snp_ids = []
    genes = []
    gene_ids = []
    pases = []
    seqs = []
    seq_vars = []
    rel_snp_poses = []
    snp_poses = []
    effect_sizes = []
    p_vals = []
    n_tissues = []
    mean_PDUIs = []
    site_types = []
    distal_snps = []
    snp_distances = []

    for snp_id in snp_dict :

        extended_snp_ids.append(snp_id)
        snp_ids.append(snp_dict[snp_id]['snp_id'])
        effect_sizes.append(np.median(snp_dict[snp_id]['effect_size']))
        p_vals.append(np.min(snp_dict[snp_id]['p_val']))
        
        genes.append(snp_dict[snp_id]['gene'])
        gene_ids.append(snp_dict[snp_id]['gene_id'])
        pases.append(snp_dict[snp_id]['pas'])
        seqs.append(snp_dict[snp_id]['wide_seq_ext'])
        seq_vars.append(snp_dict[snp_id]['wide_seq_ext_var'])
        rel_snp_poses.append(snp_dict[snp_id]['rel_snp_pos'])
        snp_poses.append(snp_dict[snp_id]['snp_pos'])
        mean_PDUIs.append(snp_dict[snp_id]['mean_PDUI'])
        site_types.append(snp_dict[snp_id]['site_type'])
        distal_snps.append(snp_dict[snp_id]['distal_snp'])
        snp_distances.append(snp_dict[snp_id]['snp_distance'])

        n_tissues.append(len(snp_dict[snp_id]['snp_id']))

    median_df = pd.DataFrame({
        'extended_snp_id' : extended_snp_ids,
        'snp_id' : snp_ids,
        'gene' : genes,
        'gene_id' : gene_ids,
        'site_type' : site_types,
        'pas' : pases,
        'wide_seq_ext' : seqs,
        'wide_seq_ext_var' : seq_vars,
        'rel_snp_pos' : rel_snp_poses,
        'snp_pos' : snp_poses,
        'effect_size' : effect_sizes,
        'mean_PDUI' : mean_PDUIs,
        'p_val' : p_vals,
        'distal_snp' : distal_snps,
        'snp_distance' : snp_distances,
        'n_tissues' : n_tissues
    })
    median_df = median_df[['extended_snp_id', 'snp_id', 'gene', 'gene_id', 'site_type', 'pas', 'wide_seq_ext', 'wide_seq_ext_var', 'rel_snp_pos', 'snp_pos', 'distal_snp', 'snp_distance', 'mean_PDUI', 'p_val', 'effect_size', 'n_tissues']]

    median_df.to_csv("polyadb_median" + version_suffix + exclude_suffix + "_SNPs.csv", sep='\t')


Excluding ['']...
Processing tissue = 'Adipose_Subcutaneous'
Processing tissue = 'Adipose_Visceral_Omentum'
Processing tissue = 'Adrenal_Gland'
Processing tissue = 'Artery_Aorta'
Processing tissue = 'Artery_Coronary'
Processing tissue = 'Artery_Tibial'
Processing tissue = 'Brain_Amygdala'
Processing tissue = 'Brain_Anterior_cingulate_cortex_BA24'
Processing tissue = 'Brain_Caudate_basal_ganglia'
Processing tissue = 'Brain_Cerebellar_Hemisphere'
Processing tissue = 'Brain_Cerebellum'
Processing tissue = 'Brain_Cortex'
Processing tissue = 'Brain_Frontal_Cortex_BA9'
Processing tissue = 'Brain_Hippocampus'
Processing tissue = 'Brain_Hypothalamus'
Processing tissue = 'Brain_Nucleus_accumbens_basal_ganglia'
Processing tissue = 'Brain_Putamen_basal_ganglia'
Processing tissue = 'Brain_Spinal_cord_cervical_c-1'
Processing tissue = 'Brain_Substantia_nigra'
Processing tissue = 'Breast_Mammary_Tissue'
Processing tissue = 'Cells_EBV-transformed_lymphocytes'
Processing tissue = 'Cells_Cultured_fibro

In [13]:
median_df

,extended_snp_id,snp_id,gene,gene_id,site_type,pas,wide_seq_ext,wide_seq_ext_var,rel_snp_pos,snp_pos,distal_snp,snp_distance,mean_PDUI,p_val,effect_size,n_tissues
0,chr10_100274701_A_G_b38__BLOC1S2.10__NM_001282439,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.10,3_most_exon,3,TTTTCCATAATATTGCAAATAACATAAAATCTTAAAATAATTCCAA...,TTTTCCATAATATTGCAAATAACATAAAATCTTAAAATAATTCCAA...,259,154,-1,1408,0.036661,2.130000e-95,1.596836,23
1,chr10_100274701_A_G_b38__BLOC1S2.10__NM_001282436,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.10,3_most_exon,3,TTTTCCATAATATTGCAAATAACATAAAATCTTAAAATAATTCCAA...,TTTTCCATAATATTGCAAATAACATAAAATCTTAAAATAATTCCAA...,259,154,-1,1408,0.036695,2.130000e-95,1.597371,23
2,chr10_100274701_A_G_b38__BLOC1S2.11__NM_001282439,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.11,3_most_exon,-1,TCTTAAAATAATTCCAAGCTGAGTCTTCTAGATTGAGCAGAAATGG...,TCTTAAAATAATTCCAAGCTGAGTCTTCTAGATTGAGCAGAAATGG...,230,125,-1,1408,0.036661,2.130000e-95,1.596836,23
3,chr10_100274701_A_G_b38__BLOC1S2.9__NM_001282436,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.9,3_most_exon,-1,AAACTTGTTCTTTGCAGCTTATCACCTTGTGAATGTCGGTAACTTA...,AAACTTGTTCTTTGCAGCTTATCACCTTGTGAATGTCGGTAACTTA...,306,201,-1,1408,0.036695,2.130000e-95,1.597371,23
4,chr10_100274701_A_G_b38__BLOC1S2.12__NM_001282439,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.12,3_most_exon,0,GCAGAAATGGTGAAAGGAGTATTGATAACTTGGCGTATGTGATGGG...,GCAGAAATGGTGAAAGGAGTATTGATAACTTGGCGTATGTGATGGG...,194,89,-1,1408,0.036661,2.130000e-95,1.596836,23
5,chr10_100274701_A_G_b38__BLOC1S2.12__NM_001282436,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.12,3_most_exon,0,GCAGAAATGGTGAAAGGAGTATTGATAACTTGGCGTATGTGATGGG...,GCAGAAATGGTGAAAGGAGTATTGATAACTTGGCGTATGTGATGGG...,194,89,-1,1408,0.036695,2.130000e-95,1.597371,23
6,chr10_100274701_A_G_b38__BLOC1S2.9__NM_173809,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.9,3_most_exon,-1,AAACTTGTTCTTTGCAGCTTATCACCTTGTGAATGTCGGTAACTTA...,AAACTTGTTCTTTGCAGCTTATCACCTTGTGAATGTCGGTAACTTA...,306,201,-1,1408,0.036695,7.464000e-96,1.594780,23
7,chr10_100274701_A_G_b38__BLOC1S2.11__NM_173809,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.11,3_most_exon,-1,TCTTAAAATAATTCCAAGCTGAGTCTTCTAGATTGAGCAGAAATGG...,TCTTAAAATAATTCCAAGCTGAGTCTTCTAGATTGAGCAGAAATGG...,230,125,-1,1408,0.036695,7.464000e-96,1.594780,23
8,chr10_100274701_A_G_b38__BLOC1S2.11__NM_001282436,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.11,3_most_exon,-1,TCTTAAAATAATTCCAAGCTGAGTCTTCTAGATTGAGCAGAAATGG...,TCTTAAAATAATTCCAAGCTGAGTCTTCTAGATTGAGCAGAAATGG...,230,125,-1,1408,0.036695,2.130000e-95,1.597371,23
9,chr10_100274701_A_G_b38__BLOC1S2.9__NM_001282439,chr10_100274701_A_G_b38,BLOC1S2,BLOC1S2.9,3_most_exon,-1,AAACTTGTTCTTTGCAGCTTATCACCTTGTGAATGTCGGTAACTTA...,AAACTTGTTCTTTGCAGCTTATCACCTTGTGAATGTCGGTAACTTA...,306,201,-1,1408,0.036661,2.130000e-95,1.596836,23


In [14]:

len(median_df['snp_id'].unique())


16447

In [15]:
len(tissue_types)

49

In [16]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1")['snp_id'].unique())


6549

In [7]:
median_df = pd.read_csv("polyadb_median_v8_w_effect_sizes_SNPs.csv", sep='\t')

In [10]:

lead_median_df = median_df.copy()

lead_median_df['cut_to_snp'] = np.abs(lead_median_df['snp_pos'] - 100)
lead_median_df = lead_median_df.sort_values(by='cut_to_snp', ascending=True).drop_duplicates(subset=['snp_id'], keep='first')

blacklist_ids = [
    'chr5_76728085_C_T'
]

lead_median_df = lead_median_df.loc[~lead_median_df['snp_id'].isin(blacklist_ids)]

len(lead_median_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1")['snp_id'].unique())


6549

In [190]:

merged_df = pd.read_csv("polyadb_merged_v8_w_effect_sizes_SNPs.csv", sep='\t')


In [9]:

merged_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1 and site_type == 'Intron'")[['snp_id', 'tissue_type', 'gene_id', 'effect_size']]


,snp_id,tissue_type,gene_id,effect_size
125,chr10_12169762_A_T_b38,Adipose_Subcutaneous,NUDT5.12,-0.444448
126,chr10_12169762_A_T_b38,Adipose_Subcutaneous,NUDT5.12,-0.444448
130,chr10_12170187_T_A_b38,Adipose_Subcutaneous,NUDT5.11,0.430083
131,chr10_12170187_T_A_b38,Adipose_Subcutaneous,NUDT5.11,0.430083
133,chr10_12185896_C_A_b38,Adipose_Subcutaneous,NUDT5.4,-0.300405
134,chr10_12185896_C_A_b38,Adipose_Subcutaneous,NUDT5.4,-0.300405
136,chr10_12193494_A_G_b38,Adipose_Subcutaneous,NUDT5.1,0.149270
137,chr10_12193494_A_G_b38,Adipose_Subcutaneous,NUDT5.1,0.149270
144,chr10_124020844_A_C_b38,Adipose_Subcutaneous,CHST15.7,0.265669
151,chr10_13118035_G_A_b38,Adipose_Subcutaneous,OPTN.2,-0.679100


In [10]:

merged_df_intron = merged_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1 and site_type == 'Intron'").copy().reset_index(drop=True)


In [191]:

merged_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1 and site_type == '3_most_exon'")[['snp_id', 'tissue_type', 'gene_id', 'effect_size']]


,snp_id,tissue_type,gene_id,effect_size
4,chr10_100274701_A_G_b38,Adipose_Subcutaneous,BLOC1S2.12,1.600594
5,chr10_100274701_A_G_b38,Adipose_Subcutaneous,BLOC1S2.12,1.600260
10,chr10_100274701_A_G_b38,Adipose_Subcutaneous,BLOC1S2.12,1.599454
12,chr10_100274717_T_C_b38,Adipose_Subcutaneous,BLOC1S2.12,1.599454
14,chr10_100274717_T_C_b38,Adipose_Subcutaneous,BLOC1S2.12,1.600260
19,chr10_100274717_T_C_b38,Adipose_Subcutaneous,BLOC1S2.12,1.600594
37,chr10_1019000_C_T_b38,Adipose_Subcutaneous,GTPBP4.19,0.566579
39,chr10_1020019_C_T_b38,Adipose_Subcutaneous,GTPBP4.22,-0.580893
40,chr10_1020019_C_T_b38,Adipose_Subcutaneous,GTPBP4.21,-0.580893
42,chr10_1020031_C_A_b38,Adipose_Subcutaneous,GTPBP4.22,-0.580215


In [193]:

merged_df_utr3 = merged_df.query("snp_pos >= 20 and snp_pos < 126 and pas != -1 and site_type == '3_most_exon'").copy().reset_index(drop=True)


In [11]:

gene_names = sorted(merged_df_intron['gene'].unique().tolist())

print(str(gene_names).replace("'", "\""))


["A1BG", "AAGAB", "AARS2", "AASDH", "ABCA12", "ABCB8", "ABCC3", "ABCC5", "ABCF2", "ABHD12", "ABHD18", "ABHD5", "ABLIM3", "ACACB", "ACADM", "ACAP2", "ACO1", "ACOT13", "ACOX3", "ACPP", "ACSF3", "ACSL3", "ACTG1", "ACTR10", "ACYP1", "ADAM10", "ADAM19", "ADCY1", "ADGRA3", "ADGRL2", "ADH5", "ADRA1A", "AEBP2", "AFDN", "AFG3L1P", "AGGF1", "AGO2", "AGPAT3", "AGPS", "AGTRAP", "AHCY", "AHCYL2", "AHRR", "AIMP1", "AKR7A2", "ALPK1", "AMFR", "ANGPTL1", "ANKIB1", "ANKMY1", "ANKRD10", "ANKRD12", "ANKRD18CP", "ANKRD42", "ANKRD44", "ANKS1B", "ANKS6", "ANO10", "ANTXR2", "ANXA11", "AP1G1", "AP1M1", "AP2B1", "AP3B1", "APIP", "APOL1", "APPL2", "AQR", "ARAF", "ARFGEF2", "ARHGAP12", "ARHGEF19", "ARHGEF38", "ARID4B", "ARIH2", "ARL14EP", "ARL16", "ARL5A", "ARMC9", "ARMCX4", "ARMCX5-GPRASP2", "ARPC4-TTLL3", "ARPC5", "ARSG", "ARSJ", "ASAH1", "ASPH", "ATF6", "ATG13", "ATG2B", "ATOH8", "ATP2C1", "ATP6V1C2", "ATP7B", "ATPAF1", "AVL9", "BARD1", "BCAT1", "BCDIN3D", "BCKDHB", "BCL2L13", "BCR", "BEST3", "BICD1", "BID", "

In [20]:

json_str = !wget -q --header='Content-type:application/json' --header='Accept:application/json' --post-data='{ "symbols" : ["A1BG", "AAGAB", "AARS2", "AASDH", "ABCA12", "ABCB8", "ABCC3", "ABCC5", "ABCF2", "ABHD12", "ABHD18", "ABHD5", "ABLIM3", "ACACB", "ACADM", "ACAP2", "ACO1", "ACOT13", "ACOX3", "ACPP", "ACSF3", "ACSL3", "ACTG1", "ACTR10", "ACYP1", "ADAM10", "ADAM19", "ADCY1", "ADGRA3", "ADGRL2", "ADH5", "ADRA1A", "AEBP2", "AFDN", "AFG3L1P", "AGGF1", "AGO2", "AGPAT3", "AGPS", "AGTRAP", "AHCY", "AHCYL2", "AHRR", "AIMP1", "AKR7A2", "ALPK1", "AMFR", "ANGPTL1", "ANKIB1", "ANKMY1", "ANKRD10", "ANKRD12", "ANKRD18CP", "ANKRD42", "ANKRD44", "ANKS1B", "ANKS6", "ANO10", "ANTXR2", "ANXA11", "AP1G1", "AP1M1", "AP2B1", "AP3B1", "APIP", "APOL1", "APPL2", "AQR", "ARAF", "ARFGEF2", "ARHGAP12", "ARHGEF19", "ARHGEF38", "ARID4B", "ARIH2", "ARL14EP", "ARL16", "ARL5A", "ARMC9", "ARMCX4", "ARMCX5-GPRASP2", "ARPC4-TTLL3", "ARPC5", "ARSG", "ARSJ", "ASAH1", "ASPH", "ATF6", "ATG13", "ATG2B", "ATOH8", "ATP2C1", "ATP6V1C2", "ATP7B", "ATPAF1", "AVL9", "BARD1", "BCAT1", "BCDIN3D", "BCKDHB", "BCL2L13", "BCR", "BEST3", "BICD1", "BID", "BIRC5", "BLZF1", "BMP2K", "BMS1", "BNIP2", "BPTF", "BRAF", "BRD4", "BRWD1", "BTD", "BTN3A2", "BTNL9", "BUD23", "C11orf71", "C11orf80", "C14orf132", "C15orf40", "C15orf61", "C16orf72", "C19orf47", "C1QTNF6", "C1orf21", "C21orf58", "C22orf39", "C2CD6", "C6orf89", "C7orf50", "C9orf129", "C9orf72", "CACTIN", "CALCOCO2", "CAMK2G", "CAMSAP1", "CAMTA1", "CAND1", "CAPN5", "CARD8", "CARF", "CASC3", "CASQ2", "CASZ1", "CATSPER2P1", "CAVIN4", "CBR3", "CBX5", "CC2D1B", "CCAR1", "CCDC122", "CCDC15", "CCDC157", "CCDC171", "CCDC180", "CCDC34", "CCDC50", "CCDC57", "CCDC82", "CCDC88A", "CCPG1", "CCT5", "CCT6P1", "CCZ1P-OR7E38P", "CD109", "CD164", "CD274", "CD320", "CD44", "CD47", "CD55", "CDC34", "CDC37L1", "CDC42BPA", "CDCA4", "CDH11", "CDH19", "CDK1", "CDK13", "CDK19", "CDK5RAP2", "CEBPZOS", "CELF3", "CELF5", "CENPC", "CENPN", "CENPO", "CEP104", "CEP120", "CEP70", "CEP76", "CEP83", "CEP89", "CEP95", "CERS1", "CFAP44", "CFLAR", "CGGBP1", "CHD6", "CHID1", "CHST15", "CLCN3", "CLCN6", "CLEC16A", "CLN3", "CLN5", "CLOCK", "CLUAP1", "CMC2", "CNOT1", "CNOT6", "CNOT7", "CNPY2", "CNPY3", "CNTNAP3", "CNTRL", "COA1", "COBL", "COL18A1", "COL6A2", "COMMD6", "COMT", "COPA", "COPG1", "COPG2", "COX11", "COX15", "COX20", "COX7A2L", "CPNE3", "CPNE5", "CPT1B", "CPVL", "CRBN", "CREB1", "CREBL2", "CREG1", "CROT", "CRTAP", "CSF1", "CSNK1G1", "CTBS", "CTNNA1", "CTNNB1", "CTNNBIP1", "CTNS", "CTPS2", "CTSS", "CWC27", "CXADR", "CYBA", "CYP4V2", "D2HGDH", "DAPK2", "DCAF10", "DCAF4", "DCAF5", "DCAF7", "DCN", "DDAH1", "DDHD2", "DDOST", "DDX19B", "DDX31", "DDX49", "DEF8", "DENND2D", "DENND3", "DFFB", "DHFR", "DHODH", "DHX8", "DIP2A", "DIP2C", "DISP1", "DMAC2", "DMXL2", "DNAJC24", "DPF3", "DPY19L1", "DRAM1", "DRAM2", "DSE", "DTWD1", "DUSP18", "DUSP28", "DYM", "DYNC2LI1", "DYRK4", "ECHDC1", "ECM2", "EEF1E1", "EFCAB14", "EFEMP1", "EFHC1", "EHD4", "EHMT1", "EIF2A", "EIF2S2", "EIF3H", "EIF3I", "EIF4E2", "ELAC2", "ELL2", "EMC2", "EMC3", "ENAH", "ENDOV", "ENOSF1", "ENSA", "EPB41L4A", "EPHX2", "EPM2A", "EPS15", "ERAP1", "ERCC6L2", "ERCC8", "ERG", "ERICH2", "ERO1B", "ESD", "ESF1", "ESR2", "ESYT2", "ETFA", "ETV1", "EXOC2", "EXOC3", "EXOC4", "EXOSC7", "EXTL3", "FAM114A1", "FAM118A", "FAM120A", "FAM124A", "FAM13A", "FAM149A", "FAM162A", "FAM168B", "FAM207A", "FAM219B", "FAM53B", "FAM86HP", "FANCI", "FANCM", "FAR2P1", "FBLN1", "FBRSL1", "FBXL4", "FBXO21", "FBXO34", "FBXO7", "FECH", "FEZ1", "FEZ2", "FGD4", "FGFR1OP", "FGFR1OP2", "FHAD1", "FKBP15", "FLCN", "FLT1", "FMC1", "FN1", "FNBP1", "FOXP4", "FRMD1", "FSTL1", "FXN", "FYCO1", "G3BP1", "GAA", "GABRA2", "GALNT15", "GANC", "GATM", "GCC2", "GCNT1", "GDE1", "GDI2", "GEMIN8", "GFAP", "GFM1", "GIGYF2", "GIMAP1-GIMAP5", "GIPC2", "GKAP1", "GLIPR1L2", "GLS", "GLYR1", "GM2A", "GMPS", "GNA12", "GNB5", "GNPDA2", "GOLGA8A", "GOLM1", "GOLPH3L", "GOLT1B", "GOSR2", "GOT2", "GPALPP1", "GPAT4", "GPATCH2L", "GPCPD1", "GPR107", "GPR137B", "GPR75-ASB3", "GPSM2", "GRPEL1", "GRSF1", "GRTP1", "GS1-124K5", "GSKIP", "GSTM4", "GSTO1", "GSTO2", "GTPBP10", "GUF1", "GUSBP11", "GXYLT2", "HADH", "HAGH", "HAUS7", "HBS1L", "HDAC4", "HDDC2", "HDDC3", "HDHD2", "HDLBP", "HELQ", "HIBCH", "HIGD1A", "HINFP", "HIP1R", "HJURP", "HM13", "HMCES", "HMGCLL1", "HNMT", "HNRNPUL1", "HPS1", "HPS4", "HS2ST1", "HSBP1L1", "HSD17B12", "HSD17B4", "HSP90AA1", "HSPA4", "HSPA8", "HUS1", "IFI16", "IFNAR2", "IFT140", "IFT57", "IFT74", "IGF1R", "IKBIP", "IL1RAP", "IL1RL1", "IMMP2L", "ING5", "INIP", "INPP4B", "INSIG2", "INTS7", "IRAK4", "IREB2", "ISG20", "ITGA11", "ITGB1", "ITGB3", "ITGB3BP", "IVD", "JPH2", "KAT14", "KAT6A", "KATNAL2", "KCNK3", "KCNMB3", "KCTD10", "KCTD20", "KDELR2", "KIAA1217", "KIAA1328", "KLF8", "KLHDC3", "KLHL24", "KLHL26", "KLHL5", "KLHL7", "KLK4", "KPNA1", "KRI1", "KRR1", "KRT5", "KYAT3", "KYNU", "L3HYPDH", "L3MBTL2", "LACTB2", "LAMA4", "LAMC1", "LAMC3", "LAMP2", "LARGE1", "LARP1B", "LDLR", "LDLRAD4", "LEMD2", "LGALS8", "LIG3", "LIMK2", "LINS1", "LMF1", "LOC728743", "LONP2", "LPCAT2", "LPIN1", "LRCH3", "LRP11", "LRPPRC", "LRRC23", "LRRFIP1", "LRRFIP2", "LSG1", "LSM1", "LSM6", "LTBP4", "LYN", "LYRM4", "MAATS1", "MALT1", "MAN1A2", "MAN2B2", "MAP3K20", "MAP3K7", "MAP7D3", "MAPK1IP1L", "MAPK8", "MAPK9", "MAPKAP1", "MARK3", "MATR3", "MAX", "MBNL1", "MCF2L", "MCPH1", "MCUR1", "MDGA1", "MDM2", "MEAF6", "MED17", "MED6", "MED8", "MEF2A", "MEGF8", "MEIG1", "MESD", "METTL15", "METTL21A", "MFGE8", "MFSD11", "MFSD13A", "MGAT1", "MIA2", "MIA3", "MIER1", "MIOS", "MIPOL1", "MKI67", "MKKS", "MKNK2", "MMAB", "MON2", "MPHOSPH6", "MPP3", "MPP7", "MPRIP", "MR1", "MRAS", "MRPL28", "MRPL35", "MRPL42", "MRPL48", "MRPL52", "MRPS25", "MRS2", "MS4A7", "MSL3", "MSR1", "MSRB2", "MTA3", "MTAP", "MTERF4", "MTG1", "MTG2", "MTHFD1L", "MTHFR", "MTHFSD", "MTMR1", "MTMR3", "MTO1", "MTPAP", "MTRF1L", "MTRR", "MVB12A", "MVB12B", "MXRA7", "MYBPC1", "MYL12A", "MYO18A", "MYO6", "MZT2B", "N4BP2L1", "N4BP2L2", "N6AMT1", "NAA60", "NAAA", "NACA", "NAGA", "NAP1L1", "NARF", "NARS2", "NBDY", "NBPF3", "NCAPG2", "NCK1", "NCOR1", "NDST1", "NDUFA10", "NDUFAF6", "NDUFAF7", "NDUFV3", "NEDD9", "NEK6", "NEK9", "NF1", "NFATC1", "NINL", "NIPSNAP3A", "NLRC5", "NME4", "NMRK1", "NOL10", "NOM1", "NPAS2", "NPEPPS", "NPRL3", "NR2F2", "NRG4", "NRP2", "NSD2", "NSL1", "NSUN6", "NT5DC1", "NT5DC3", "NUB1", "NUCKS1", "NUDCD3", "NUDT5", "NUFIP2", "NUP133", "NUP43", "NUP50", "NUP54" ] }' 'http://rest.ensembl.org/lookup/symbol/homo_sapiens'  -O -


In [22]:

json_str_2 = !wget -q --header='Content-type:application/json' --header='Accept:application/json' --post-data='{ "symbols" : ["NXPE3", "OARD1", "OAS2", "OBSL1", "ODF2", "ODF2L", "OGDH", "OGFOD3", "OGG1", "OLA1", "OLFML1", "OPA1", "OPA3", "OPTN", "OSBPL10", "OSMR", "OXNAD1", "PACRGL", "PAFAH1B1", "PAK2", "PAM", "PANK1", "PAPOLA", "PARP4", "PARP9", "PASK", "PAWR", "PAX8", "PCBP3", "PCIF1", "PCM1", "PCMTD1", "PCNX4", "PCYT1A", "PDCD2", "PDCD6IP", "PDE1A", "PDK1", "PDK2", "PDLIM1", "PDLIM3", "PDS5B", "PDXDC2P-NPIPB14P", "PDZD7", "PEPD", "PERP", "PEX13", "PFKP", "PGM3", "PGS1", "PHC2", "PHF20L1", "PHLDB3", "PHTF1", "PIAS2", "PIBF1", "PIGC", "PIGG", "PIGK", "PIGN", "PIGP", "PIN4", "PINK1", "PKN2", "PKNOX1", "PLAT", "PLBD1", "PLBD2", "PLCD3", "PLEKHA6", "PLEKHG2", "PMS1", "PMS2", "PNPLA8", "PNPT1", "POGK", "POLA2", "POLI", "POLR1D", "POMT1", "POT1", "POU2F3", "POU6F1", "PPFIA3", "PPHLN1", "PPIE", "PPIL2", "PPIL6", "PPM1K", "PPP1R1C", "PPP2R2A", "PPP2R2D", "PPP6C", "PPP6R2", "PRCD", "PRCP", "PRDM15", "PRDM8", "PRELID3B", "PREP", "PRKAR1A", "PRKCH", "PRKD3", "PRMT2", "PRMT7", "PROM2", "PRPSAP1", "PRSS12", "PRSS23", "PSCA", "PSEN1", "PSENEN", "PSMD11", "PSMD6", "PSMF1", "PSMG4", "PSPC1", "PTBP2", "PTBP3", "PTCD3", "PTGER3", "PTGFRN", "PTGR2", "PTGS1", "PTPN1", "PTPN18", "PTPN21", "PTPRE", "PVR", "PWP1", "PWWP2A", "PXMP2", "PYGB", "PYGL", "QDPR", "QKI", "QRSL1", "RAB10", "RAB11A", "RAB22A", "RAB29", "RAB2A", "RAB3GAP1", "RAB3IP", "RAB5A", "RAB5B", "RABEP1", "RABGAP1", "RABL2B", "RAD23B", "RALBP1", "RALGAPA1", "RALY", "RANGAP1", "RAP1A", "RAP1GDS1", "RAPGEF3", "RARRES1", "RARS2", "RBM18", "RBM19", "RBM3", "RCBTB1", "RCCD1", "RCN1", "RDH10", "RDX", "REEP1", "REEP5", "RET", "REV3L", "RFC3", "RGL1", "RGS12", "RGS3", "RGS5", "RIC1", "RIC3", "RIF1", "RIPOR2", "RMDN2", "RNF41", "RNLS", "RNMT", "ROCK2", "ROR1", "RPAP2", "RPF1", "RPGRIP1L", "RPL15", "RPL27A", "RPL28", "RPL31", "RPRD1A", "RPS20", "RPS6KB1", "RTKN2", "RUFY3", "RUVBL1", "RYK", "SART1", "SC5D", "SCAPER", "SCD5", "SCP2", "SDHA", "SDHAF4", "SEC13", "SEC14L1", "SEC14L2", "SEC22A", "SEC63", "SEMA4B", "SENP5", "SENP6", "SERINC3", "SERP1", "SERPINB1", "SERPINI2", "SETD7", "SETD9", "SETMAR", "SGO1", "SGPL1", "SGPP2", "SH3BP5", "SH3RF2", "SHPK", "SIRT3", "SLC11A2", "SLC13A3", "SLC15A2", "SLC16A4", "SLC20A2", "SLC25A16", "SLC26A1", "SLC2A8", "SLC30A9", "SLC31A1", "SLC35B3", "SLC35B4", "SLC35E2B", "SLC35F4", "SLC35F5", "SLC35G2", "SLC36A1", "SLC38A10", "SLC39A9", "SLC44A1", "SLC48A1", "SLC6A15", "SLC6A2", "SLCO3A1", "SLMAP", "SMAD2", "SMARCE1", "SMC2", "SMC3", "SMC6", "SMG9", "SMIM19", "SMIM8", "SMOC1", "SMURF2", "SNAPC1", "SNAPC3", "SNED1", "SNRK", "SNX13", "SNX16", "SNX19", "SNX21", "SNX5", "SOD2", "SORT1", "SP110", "SP3", "SPART", "SPATA9", "SPCS3", "SPDL1", "SPECC1", "SPEF2", "SPG7", "SPOCK3", "SPPL2A", "SPPL2B", "SPRED1", "SPTBN1", "SPTLC1", "SPTLC3", "SRD5A1", "SRD5A3", "SRP19", "SRP72", "SSBP3", "SSR1", "ST6GALNAC2", "ST6GALNAC3", "ST7L", "STAG1", "STAG3L4", "STARD3", "STARD9", "STC2", "STEAP3", "STIM1", "STK17A", "STK33", "STK39", "STMP1", "STPG1", "STRADB", "STRAP", "STT3A", "STX17", "SUCLG2", "SUGP2", "SULF1", "SULF2", "SULT4A1", "SUN2", "SUPT3H", "SVIL", "SVIP", "SYCE1", "SYMPK", "SYN2", "SYNE3", "SYNJ2BP-COX16", "SYS1", "SYVN1", "TAF3", "TAF8", "TANK", "TAPT1", "TARDBP", "TARP", "TATDN2", "TATDN3", "TBC1D20", "TBC1D2B", "TBCA", "TBCK", "TBL1XR1", "TCEANC2", "TCFL5", "TCTN3", "TDRP", "TECPR2", "TEKT4P2", "TES", "TFB1M", "TGFBRAP1", "TGS1", "THBS2", "THEM4", "THOC5", "TIMM23B", "TIMMDC1", "TK2", "TKT", "TMBIM4", "TMCO3", "TMCO4", "TMED4", "TMEM107", "TMEM120A", "TMEM135", "TMEM167A", "TMEM177", "TMEM185A", "TMEM218", "TMEM220", "TMEM230", "TMEM255B", "TMEM38B", "TMEM39A", "TMEM41B", "TMEM8B", "TMEM91", "TMLHE", "TMPO", "TMPRSS3", "TMTC4", "TMX4", "TNFSF10", "TNNT1", "TOMM70", "TOPBP1", "TOR1A", "TOR1B", "TPGS2", "TPM1", "TPMT", "TPP2", "TPST2", "TPTE2P1", "TRABD", "TRAK1", "TRAPPC10", "TRDN", "TRIM13", "TRIM33", "TRIM4", "TRIM7", "TRIOBP", "TRIP11", "TRNAU1AP", "TRNT1", "TRPM7", "TSNARE1", "TSPAN11", "TSPAN14", "TSPAN3", "TTC12", "TTC33", "TTC39B", "TTL", "TUBGCP3", "TULP3", "TUSC3", "TWSG1", "TXN", "TXNDC5", "TXNRD2", "TYW1", "TYW1B", "U2SURP", "UBAP2", "UBAP2L", "UBE2E2", "UBE2G2", "UBE2R2", "UBE2W", "UBE3A", "UBE3D", "UBIAD1", "UBOX5", "UBQLN1", "UCHL3", "UFD1", "UFSP2", "UGGT1", "UMAD1", "UMPS", "UQCC2", "UQCRB", "URI1", "USP12", "USP13", "USP14", "USP40", "USP45", "USP47", "USP48", "USP6NL", "UVSSA", "VAMP4", "VAT1L", "VAV3", "VCAN", "VEGFC", "VHL", "VIM", "VIRMA", "VIT", "VKORC1L1", "VPS13A", "VPS26A", "VPS33A", "VPS37A", "VPS41", "WAC", "WARS2", "WASF2", "WASHC2A", "WDPCP", "WDR19", "WDR33", "WDR35", "WDR4", "WDR60", "WDR92", "WEE1", "WIPF1", "WRAP53", "WTAP", "WWP2", "XPA", "XPR1", "XRN2", "YBEY", "YIPF7", "YTHDC2", "YWHAZ", "ZBTB1", "ZBTB25", "ZBTB44", "ZC2HC1A", "ZC3H13", "ZCCHC7", "ZDHHC14", "ZDHHC9", "ZFAND4", "ZFP28", "ZFP36L1", "ZNF12", "ZNF138", "ZNF141", "ZNF175", "ZNF232", "ZNF267", "ZNF273", "ZNF276", "ZNF280D", "ZNF320", "ZNF330", "ZNF333", "ZNF33B", "ZNF346", "ZNF37A", "ZNF398", "ZNF462", "ZNF480", "ZNF500", "ZNF516", "ZNF532", "ZNF562", "ZNF577", "ZNF592", "ZNF641", "ZNF652", "ZNF717", "ZNF720", "ZNF83", "ZNF84", "ZNF846", "ZNF880", "ZNHIT6", "ZNRF1", "ZP3", "ZRANB3", "ZSWIM7", "ZXDC", "ZZEF1" ] }' 'http://rest.ensembl.org/lookup/symbol/homo_sapiens'  -O -


In [23]:
import json

ens_dict = json.loads(json_str[0])


In [24]:
import json

ens_dict_2 = json.loads(json_str_2[0])


In [26]:

ens_ids = []
for _, row in merged_df_intron.iterrows() :
    if row['gene'] in ens_dict :
        ens_ids.append(ens_dict[row['gene']]['id'])
    elif row['gene'] in ens_dict_2 :
        ens_ids.append(ens_dict_2[row['gene']]['id'])
    else :
        ens_ids.append("Missing")

merged_df_intron['ens_id'] = ens_ids


In [194]:

gene_names = sorted(merged_df_utr3['gene'].unique().tolist())

print(str(gene_names).replace("'", "\""))


["A1BG", "AAGAB", "AARS2", "ABAT", "ABCA5", "ABCB8", "ABHD12", "ABHD6", "ABI1", "ABLIM1", "ABLIM2", "ACAA2", "ACACB", "ACADM", "ACADSB", "ACCS", "ACER3", "ACHE", "ACO1", "ACOT13", "ACOX1", "ACOX3", "ACOXL", "ACRBP", "ACSBG2", "ACSF3", "ACSS3", "ACTB", "ACTG1", "ACTR1A", "ACTR2", "ACTR8", "ACVRL1", "ADAL", "ADAM10", "ADAM11", "ADAM22", "ADARB2", "ADAT1", "ADCY1", "ADD2", "ADGRA3", "ADGRE2", "ADGRG1", "ADGRL4", "ADH5", "ADK", "ADNP", "AEBP2", "AFAP1", "AFDN", "AFG3L1P", "AGAP2", "AGFG1", "AGGF1", "AGO2", "AGPAT3", "AGPAT4", "AGR2", "AGT", "AGTRAP", "AGXT", "AHCY", "AHRR", "AIFM2", "AIMP1", "AJUBA", "AK2", "AK3", "AKAP1", "AKAP11", "AKAP12", "AKAP13", "AKIRIN1", "AKIRIN2", "AKR1C1", "AKR1C2", "AKR1C3", "AKR1C4", "AKTIP", "ALCAM", "ALDH16A1", "ALDH1A3", "ALDH6A1", "ALDH7A1", "ALDOB", "ALKAL2", "ALKBH1", "ALMS1", "ALMS1P1", "ALPK3", "ALS2CL", "AMFR", "AMHR2", "AMIGO1", "AMIGO2", "AMOTL1", "AMZ2", "ANAPC16", "ANGEL2", "ANGPT2", "ANK3", "ANKH", "ANKIB1", "ANKLE2", "ANKRD10", "ANKRD12", "ANKRD

In [195]:

json_str = !wget -q --header='Content-type:application/json' --header='Accept:application/json' --post-data='{ "symbols" : ["A1BG", "AAGAB", "AARS2", "ABAT", "ABCA5", "ABCB8", "ABHD12", "ABHD6", "ABI1", "ABLIM1", "ABLIM2", "ACAA2", "ACACB", "ACADM", "ACADSB", "ACCS", "ACER3", "ACHE", "ACO1", "ACOT13", "ACOX1", "ACOX3", "ACOXL", "ACRBP", "ACSBG2", "ACSF3", "ACSS3", "ACTB", "ACTG1", "ACTR1A", "ACTR2", "ACTR8", "ACVRL1", "ADAL", "ADAM10", "ADAM11", "ADAM22", "ADARB2", "ADAT1", "ADCY1", "ADD2", "ADGRA3", "ADGRE2", "ADGRG1", "ADGRL4", "ADH5", "ADK", "ADNP", "AEBP2", "AFAP1", "AFDN", "AFG3L1P", "AGAP2", "AGFG1", "AGGF1", "AGO2", "AGPAT3", "AGPAT4", "AGR2", "AGT", "AGTRAP", "AGXT", "AHCY", "AHRR", "AIFM2", "AIMP1", "AJUBA", "AK2", "AK3", "AKAP1", "AKAP11", "AKAP12", "AKAP13", "AKIRIN1", "AKIRIN2", "AKR1C1", "AKR1C2", "AKR1C3", "AKR1C4", "AKTIP", "ALCAM", "ALDH16A1", "ALDH1A3", "ALDH6A1", "ALDH7A1", "ALDOB", "ALKAL2", "ALKBH1", "ALMS1", "ALMS1P1", "ALPK3", "ALS2CL", "AMFR", "AMHR2", "AMIGO1", "AMIGO2", "AMOTL1", "AMZ2", "ANAPC16", "ANGEL2", "ANGPT2", "ANK3", "ANKH", "ANKIB1", "ANKLE2", "ANKRD10", "ANKRD12", "ANKRD26", "ANKRD42", "ANKS6", "ANXA11", "ANXA2", "ANXA7", "AOX1", "AP1M1", "AP1M2", "AP1S1", "AP5M1", "AP5Z1", "APBA3", "APBB2", "APEH", "APLF", "APOBEC3C", "APOBEC3D", "APOBEC3F", "APOC2", "APOD", "APOL2", "APOL4", "APRT", "APTX", "AQP3", "AQR", "ARF4", "ARHGAP12", "ARHGAP18", "ARHGAP24", "ARHGAP28", "ARHGAP29", "ARHGAP31", "ARHGAP40", "ARHGAP8", "ARHGDIG", "ARHGEF17", "ARHGEF19", "ARL14EP", "ARL16", "ARL4A", "ARL5A", "ARL6IP5", "ARNTL2", "ARPC1B", "ARPC2", "ARPC5", "ARPC5L", "ARPP19", "ARRDC2", "ARSA", "ARSJ", "ASAH1", "ASAH2B", "ASB1", "ASB6", "ASB8", "ASCC1", "ASCC3", "ASL", "ASPG", "ASPM", "ASXL1", "ATF6", "ATG10", "ATG12", "ATG2B", "ATL3", "ATP13A4", "ATP1B3", "ATP6AP1", "ATP6V1C1", "ATP6V1C2", "ATP6V1G1", "ATP8A1", "ATXN3", "ATXN7L3B", "AUNIP", "AVL9", "AZIN1", "B3GLCT", "B3GNT3", "B3GNT9", "B4GALT1", "B4GALT4", "BAG1", "BAG2", "BAG3", "BAK1", "BARD1", "BAX", "BCAS1", "BCDIN3D", "BCL2L13", "BCLAF3", "BET1L", "BGN", "BHMT2", "BICD1", "BICDL2", "BID", "BIRC3", "BIRC5", "BLOC1S2", "BMPER", "BMPR1A", "BMS1", "BMS1P20", "BNIP3", "BOK", "BORCS7", "BRD7", "BRK1", "BRWD1", "BST1", "BST2", "BTAF1", "BTBD9", "BTD", "BTF3", "BTF3L4", "BTN3A2", "BUD23", "C11orf54", "C11orf58", "C11orf91", "C12orf43", "C12orf45", "C12orf73", "C12orf75", "C14orf132", "C16orf72", "C17orf100", "C17orf80", "C19orf25", "C19orf47", "C19orf53", "C1GALT1", "C1QB", "C1QTNF2", "C1orf21", "C1orf43", "C3orf14", "C5orf38", "C6orf89", "C7", "C9orf72", "C9orf78", "CA12", "CACNA1B", "CACNA1E", "CACUL1", "CADM3", "CALCOCO1", "CALCOCO2", "CALCRL", "CALU", "CAMK1D", "CAMK2D", "CAMSAP1", "CAND1", "CANT1", "CAPS", "CAPZA2", "CARD14", "CARD19", "CARD8", "CARF", "CASK", "CASQ2", "CAST", "CAV2", "CAVIN4", "CC2D1B", "CCAR1", "CCBE1", "CCDC127", "CCDC149", "CCDC157", "CCDC171", "CCDC174", "CCDC180", "CCDC184", "CCDC25", "CCDC43", "CCDC50", "CCDC57", "CCDC6", "CCDC68", "CCDC85B", "CCDC85C", "CCDC88A", "CCDC90B", "CCNI", "CCNJL", "CCNY", "CCPG1", "CCSAP", "CCT5", "CCZ1P-OR7E38P", "CD151", "CD164", "CD177", "CD226", "CD300C", "CD36", "CD40", "CD47", "CD55", "CD59", "CD81", "CDC42BPA", "CDC42BPB", "CDC42EP1", "CDC42EP3", "CDC7", "CDCA4", "CDH11", "CDH3", "CDHR3", "CDK1", "CDK12", "CDK13", "CDK5RAP2", "CDK5RAP3", "CDK9", "CDKAL1", "CDS2", "CDT1", "CDV3", "CEACAM5", "CEBPG", "CEBPZOS", "CELF1", "CENPBD1P1", "CENPC", "CENPN", "CENPO", "CEP104", "CEP250", "CEP350", "CEP41", "CEP57", "CEP70", "CEP89", "CERS1", "CERS2", "CERS5", "CETN2", "CFAP206", "CFAP74", "CFAP97", "CFDP1", "CFL2", "CFLAR", "CGB3", "CHCHD2", "CHCHD7", "CHD6", "CHDH", "CHIC2", "CHKB", "CHML", "CHN2", "CHRAC1", "CHRNE", "CHST1", "CHST11", "CHST15", "CHST6", "CHST7", "CHSY1", "CHTOP", "CIAO1", "CIR1", "CISD1", "CISD2", "CKAP2", "CKAP5", "CKB", "CKMT1B", "CLCN3", "CLCN7", "CLDN1", "CLEC2B", "CLEC2D", "CLEC4D", "CLEC7A", "CLIC3", "CLMN", "CLN5", "CLNS1A", "CLOCK", "CLPB", "CLPP", "CLTB", "CLUAP1", "CMBL", "CMC2", "CMPK1", "CMYA5", "CNDP2", "CNIH1", "CNIH4", "CNKSR3", "CNNM2", "CNNM3", "CNOT1", "CNOT6", "CNOT7", "CNP", "CNPY2", "CNPY3", "CNTN2", "CNTROB", "COBL", "COBLL1", "COCH", "COL1A1", "COL25A1", "COL28A1", "COL4A2", "COL5A1", "COL5A2", "COL6A6", "COL8A1", "COL9A3", "COLCA1", "COLEC12", "COMMD1", "COMMD2", "COMMD5", "COMMD6", "COMMD9", "COMT", "COMTD1", "COPB2", "COPS2", "COPS6", "COPS8", "COQ6", "COQ7", "COQ8A", "CORO2A", "COX19", "COX20", "COX5A", "COX6A1", "CPA3", "CPD", "CPNE2", "CPNE3", "CPSF2", "CPSF6", "CPVL", "CRBN", "CREB1", "CREBL2", "CREBZF", "CREG1", "CREM", "CRIPT", "CRISPLD1", "CRKL", "CRLS1", "CROCCP3", "CRTAP", "CRYZ", "CRYZL1", "CSDE1", "CSE1L", "CSGALNACT2", "CSKMT", "CSNK1A1", "CSNK1D", "CSNK1G1", "CSNK2A1", "CST7", "CSTF2T", "CTC1", "CTNNA1", "CTNNB1", "CTNNBIP1", "CTNS", "CTSD", "CTSF", "CTSS", "CTSV", "CTSZ", "CUL3", "CWC15", "CYB5A", "CYB5B", "CYB5R2", "CYBA", "CYBRD1", "CYCS", "CYP2C18", "CYP2J2", "CYP4A11", "CYP4F11", "CYP4F24P", "CYP4F3", "CYP4F8", "CYP4V2", "CYTH2", "CYTIP", "D2HGDH", "DAD1", "DAPK2", "DAZL", "DBF4B", "DBNL", "DCAF10", "DCBLD2", "DCLK1", "DCP2", "DCTD", "DCTN5", "DCUN1D2", "DCUN1D3", "DCUN1D4", "DDHD2", "DDO", "DDOST", "DDX18", "DDX19B", "DDX20", "DDX27", "DDX31", "DDX58", "DEGS1", "DEGS2", "DENND2D", "DENND3", "DENND6B", "DEPTOR", "DERL1", "DFFB", "DHFR", "DHODH", "DHRS12", "DHX34", "DIP2A", "DIP2B", "DIS3L", "DLAT", "DLD", "DLG1", "DNAJA1", "DNAJC15", "DNAJC25", "DNAJC30", "DNALI1", "DNM1L", "DNPEP", "DNTTIP2", "DOCK2", "DOCK9", "DPH3", "DPP10", "DPP8", "DPY19L1", "DPY19L2P4", "DPY30", "DPYSL4", "DR1", "DRAM2", "DROSHA", "DSC2", "DSCC1", "DSE", "DSG3", "DSTN", "DTD2", "DTWD1", "DUOXA1", "DUSP28", "DYNC1H1", "DYNC1I2", "DYNLL2", "DYRK1A", "DZIP1", "E2F6", "EARS2", "EBNA1BP2", "ECD", "ECHDC1", "ECI2", "EDNRA", "EEF1A1", "EEF1AKMT2", "EEPD1", "EFCAB11", "EFCAB14", "EFCAB2", "EFHC1", "EFTUD2", "EGFR", "EGLN3", "EHD3", "EHD4", "EI24", "EIF1", "EIF1AD", "EIF2A" ] }' 'http://rest.ensembl.org/lookup/symbol/homo_sapiens'  -O -


In [196]:

json_str_2 = !wget -q --header='Content-type:application/json' --header='Accept:application/json' --post-data='{ "symbols" : ["EIF2B1", "EIF2B2", "EIF2S3", "EIF3F", "EIF3H", "EIF3I", "EIF3M", "EIF4B", "EIF4E", "EIF4E2", "EIF4E3", "EIF4EBP2", "EIF5", "EIF5A2", "EIF6", "ELAC2", "ELOVL4", "EMC1", "EMC2", "EMC3", "EMCN", "EMP2", "EMP3", "EMSY", "ENAH", "ENDOD1", "ENDOV", "ENGASE", "ENOSF1", "ENPEP", "ENPP1", "ENPP4", "ENPP5", "ENSA", "ENTPD6", "ENTPD7", "EP400", "EPB41L1", "EPCAM", "EPHA4", "EPS15", "ERAP1", "ERCC4", "ERCC6", "ERCC8", "ERGIC2", "ERMAP", "ERO1A", "ERO1B", "ERP29", "ERV3-1-ZNF117", "ESRP1", "ESYT2", "ETFA", "ETFDH", "ETS2", "ETV1", "EVA1C", "EVC", "EXO1", "EXOC2", "EXOC3L2", "EXOC5", "EXOC6B", "EXOG", "EXOSC7", "EXT2", "EXTL3", "F11R", "F5", "FAAP100", "FAAP24", "FABP4", "FAH", "FAIM2", "FAM110B", "FAM118A", "FAM120A", "FAM120AOS", "FAM122A", "FAM124A", "FAM126A", "FAM160B2", "FAM161B", "FAM162A", "FAM168B", "FAM177A1", "FAM193A", "FAM200B", "FAM20B", "FAM210A", "FAM219B", "FAM222B", "FAM227A", "FAM229B", "FAM76B", "FAM83E", "FAM86EP", "FAM86FP", "FAM98B", "FANCA", "FANCD2", "FANCI", "FARSB", "FAS", "FASTK", "FASTKD1", "FASTKD2", "FAXC", "FBLIM1", "FBLN2", "FBN1", "FBXL4", "FBXL5", "FBXO22", "FBXO25", "FBXO27", "FBXO28", "FBXO31", "FBXO38", "FBXO42", "FBXO7", "FBXO9", "FCGR1B", "FDX1", "FECH", "FEM1C", "FER", "FEZ2", "FGB", "FGD2", "FGF14", "FGFR1", "FGFR1OP", "FGG", "FHAD1", "FHIT", "FHL5", "FIBIN", "FIG4", "FIGNL1", "FKBP15", "FKBP1A", "FLCN", "FLRT3", "FLT1", "FLVCR1", "FLYWCH2", "FOSL2", "FOXF1", "FOXL1", "FOXRED2", "FRA10AC1", "FRK", "FRMD4A", "FRMD4B", "FRY", "FRZB", "FSTL1", "FSTL3", "FXN", "FXYD3", "FZD3", "G2E3", "G3BP1", "GAB1", "GABRA2", "GABRG2", "GAL3ST3", "GALC", "GALNT12", "GALNT2", "GAMT", "GAPDH", "GAR1", "GAS7", "GATA6", "GATAD1", "GATM", "GBP2", "GBP5", "GCAT", "GCC2", "GCFC2", "GCLM", "GCNT1", "GCSH", "GDI2", "GEM", "GEMIN7", "GEN1", "GFER", "GFM1", "GFPT1", "GGACT", "GGCX", "GGH", "GGPS1", "GHITM", "GID8", "GIGYF2", "GIMAP1-GIMAP5", "GIMAP4", "GIN1", "GIPC2", "GK5", "GKAP1", "GLE1", "GLG1", "GLIPR1", "GLIPR2", "GLMN", "GLRX", "GLRX5", "GLS", "GLT1D1", "GM2A", "GMEB2", "GMPS", "GNA11", "GNAI3", "GNG10", "GNL3L", "GNLY", "GNPDA2", "GNS", "GOLGA3", "GOLGA8A", "GOLGA8B", "GOLIM4", "GOLPH3L", "GOLT1B", "GOT1", "GOT2", "GPAT4", "GPATCH2", "GPATCH2L", "GPBP1", "GPC1", "GPC4", "GPCPD1", "GPD1L", "GPLD1", "GPR107", "GPR137B", "GPR155", "GPR156", "GPRC5B", "GPRC5C", "GPSM1", "GPX7", "GPX8", "GREM2", "GRPEL1", "GRSF1", "GRTP1", "GSE1", "GSKIP", "GSR", "GSTA4", "GSTM3", "GSTO1", "GTF2A2", "GTF3C3", "GTF3C4", "GTPBP10", "GTPBP4", "GUF1", "GXYLT2", "HABP4", "HACD1", "HACD4", "HADH", "HADHA", "HADHB", "HAGH", "HAL", "HAUS2", "HAUS6", "HBS1L", "HDAC2", "HDDC2", "HEBP2", "HFE", "HIBCH", "HIF1AN", "HIGD1A", "HINFP", "HLA-DQB1", "HM13", "HMCES", "HMCN1", "HMGB1", "HMGN1", "HMGN2", "HMGN3", "HNMT", "HNRNPA1L2", "HNRNPAB", "HNRNPDL", "HNRNPF", "HNRNPH3", "HNRNPK", "HNRNPLL", "HNRNPUL1", "HPGD", "HPS1", "HPSE2", "HS1BP3", "HS2ST1", "HSBP1", "HSBP1L1", "HSD17B12", "HSD17B4", "HSDL1", "HSP90AA1", "HSP90B1", "HSPA13", "HSPA4", "HSPA5", "HSPA8", "HSPBAP1", "HSPBP1", "HSPH1", "HTATIP2", "HTATSF1", "HTD2", "HYPK", "IAH1", "ICA1L", "ICE2", "ID4", "IDH3B", "IDI1", "IDS", "IER5", "IFI30", "IFIT5", "IFRD1", "IFT22", "IFT46", "IFT57", "IGF1R", "IGFBP2", "IGFBPL1", "IGFLR1", "IKBIP", "IL13RA1", "IL15", "IL17RA", "IL17RB", "IL18", "IMMP2L", "IMP4", "IMPA1", "INF2", "ING1", "ING5", "INIP", "INPP4B", "INPP5K", "INSIG2", "INSR", "INTS4P1", "INTS7", "IPO7", "IPO9", "IQCE", "IQGAP1", "IRAK1BP1", "IRAK3", "IRF1", "IRF2BP2", "IRF5", "IRF6", "ISCA1", "ISCA2", "ISCU", "ITGA1", "ITGA11", "ITGA3", "ITGA8", "ITGAM", "ITGB1", "ITGB1BP1", "ITGB5", "ITIH4", "ITIH5", "JAGN1", "JOSD2", "JPH2", "KANSL3", "KATNAL1", "KCNAB1", "KCNAB3", "KCNJ5", "KCNK6", "KCNMB4", "KCTD10", "KCTD20", "KDELR2", "KDELR3", "KDM5B", "KHNYN", "KIAA0930", "KIAA1586", "KIAA1755", "KIF11", "KIF3C", "KIN", "KLF2", "KLF5", "KLF8", "KLHDC3", "KLHL12", "KLHL24", "KLHL26", "KLHL36", "KLHL41", "KLHL5", "KLK10", "KLK7", "KNOP1", "KPNA1", "KRI1", "KRR1", "KRT10", "KRT6A", "KXD1", "KYAT1", "KYNU", "L3HYPDH", "L3MBTL2", "LACTB", "LACTB2", "LAMC1", "LAMC3", "LAMP2", "LAMTOR4", "LAPTM4B", "LARP4B", "LARP7", "LAYN", "LDAH", "LDLR", "LEMD2", "LGALS8", "LHB", "LINS1", "LIPH", "LIPT2", "LMAN2", "LMBRD1", "LMNB2", "LNPK", "LOC100289561", "LOC374443", "LOC389906", "LOC728613", "LOC728743", "LONP2", "LOX", "LPAR3", "LPCAT2", "LPGAT1", "LPL", "LRBA", "LRCH3", "LRIG2", "LRP11", "LRPAP1", "LRRC1", "LRRC27", "LRRC32", "LRRC47", "LRRC57", "LRRC59", "LRRC8C", "LRRFIP2", "LRRK1", "LSG1", "LSM4", "LSM5", "LSM6", "LSM8", "LTBP2", "LUM", "LURAP1L", "LVRN", "LYN", "LYPD5", "LYPLAL1", "LYRM2", "LYRM4", "LYRM7", "LYSMD3", "LYVE1", "MACO1", "MAF", "MAGOHB", "MALT1", "MAMSTR", "MAN1A2", "MAN2A2", "MANBA", "MANEAL", "MAP1LC3B", "MAP2K1", "MAP2K2", "MAP2K6", "MAP3K7", "MAP4", "MAPK10", "MAPK13", "MAPK1IP1L", "MAPK8", "MAPK9", "MAPKAPK5", "MARK4", "MATN2", "MATR3", "MAU2", "MAVS", "MBD2", "MBOAT1", "MBOAT2", "MBTPS2", "MC1R", "MCAT", "MCC", "MCF2L", "MCMBP", "MCTP2", "MCUB", "MDGA1", "MDH2", "MDM4", "MEAF6", "MECR", "MED10", "MED28", "MED29", "MED4", "MED6", "MED7", "MEF2A", "MEPCE", "MET", "METAP1D", "METAP2", "METRN", "METTL14", "METTL15", "METTL16", "METTL22", "METTL2B", "METTL7B", "MFN1", "MFSD11", "MFSD4B", "MGAT4A", "MGME1", "MGP", "MGST3", "MIEF1", "MIEN1", "MIER1", "MIOS", "MKI67", "MKKS", "MKNK2", "MKRN2", "MLANA", "MLEC", "MLF1", "MLH3", "MLPH", "MMAB", "MOCS2", "MOCS3", "MON2", "MORF4L1", "MPC2", "MPDZ", "MPHOSPH6", "MPI", "MPP3", "MPP6", "MPRIP", "MPZL1", "MR1", "MRAS", "MRE11", "MRFAP1", "MRI1", "MRO", "MRPL1", "MRPL13", "MRPL17", "MRPL19", "MRPL34", "MRPL35", "MRPL42", "MRPL50", "MRPL51", "MRPL52", "MRPL57", "MRPS10", "MRPS11", "MRPS14", "MRPS15", "MRPS16", "MRPS21", "MRPS22", "MRPS23", "MRPS25", "MRPS33", "MRPS7", "MRS2", "MRTO4", "MS4A7", "MSI2", "MSLN", "MSRB2", "MSRB3" ] }' 'http://rest.ensembl.org/lookup/symbol/homo_sapiens'  -O -


In [197]:

json_str_3 = !wget -q --header='Content-type:application/json' --header='Accept:application/json' --post-data='{ "symbols" : ["MST1R", "MT1E", "MT1L", "MT1M", "MTA2", "MTAP", "MTDH", "MTFR1L", "MTG2", "MTHFD2", "MTHFR", "MTHFSD", "MTIF2", "MTMR11", "MTMR2", "MTMR3", "MTO1", "MTR", "MTRF1L", "MTRR", "MTURN", "MXD4", "MXRA7", "MYADM", "MYBPH", "MYCBP", "MYH9", "MYL12A", "MYLK", "MYO1E", "MYO5B", "MYO6", "MYOZ2", "MYOZ3", "MZT1", "MZT2B", "N4BP3", "N6AMT1", "NAA10", "NAAA", "NAB1", "NACA", "NADK", "NAGA", "NAP1L1", "NAPG", "NARS2", "NAXE", "NBN", "NCAPD2", "NCK1", "NCKAP1", "NCOR1", "NDC1", "NDFIP1", "NDST1", "NDUFA5", "NDUFAF4", "NDUFAF8", "NDUFV3", "NEBL", "NEDD1", "NEDD9", "NEFL", "NEK6", "NEK9", "NEPRO", "NF1", "NFATC3", "NFE2L1", "NFIX", "NFKBIA", "NFKBIE", "NFRKB", "NGEF", "NICN1", "NIFK", "NINJ1", "NIP7", "NIT2", "NKAP", "NLN", "NLRC5", "NME4", "NMRK1", "NMT1", "NMT2", "NNMT", "NNT", "NOL10", "NOL6", "NOL9", "NOM1", "NOP10", "NOP16", "NOP56", "NOTCH3", "NOX5", "NPDC1", "NPEPL1", "NPEPPS", "NPLOC4", "NPRL3", "NRBF2", "NRDE2", "NSA2", "NSDHL", "NSFL1C", "NSUN4", "NSUN6", "NT5DC1", "NT5DC3", "NTPCR", "NUB1", "NUBP2", "NUCKS1", "NUDC", "NUDCD1", "NUDCD2", "NUDT21", "NUDT4", "NUDT5", "NUP107", "NUP37", "NUP50", "NUP54", "NXN", "NXPE3", "OAF", "OARD1", "OAS1", "ODF2L", "OGFOD1", "OGFOD2", "OGFOD3", "OGFRL1", "OLA1", "OPA1", "OPTN", "OR51E1", "ORAI2", "ORMDL1", "OSBPL5", "OSBPL6", "OSGEP", "OSGIN2", "OSTM1", "OTUB1", "OVOL1", "OXA1L", "OXR1", "P2RY2", "P3H3", "PACSIN2", "PAICS", "PAIP2B", "PAM", "PANK2", "PANX1", "PANX2", "PAOX", "PAQR7", "PAQR8", "PARD3", "PARG", "PARM1", "PARP1", "PARP9", "PARVA", "PARVG", "PASD1", "PAWR", "PAX8", "PBLD", "PBXIP1", "PCBP2", "PCBP3", "PCDH18", "PCDHB2", "PCM1", "PCMT1", "PCNX1", "PCNX4", "PCSK2", "PCSK4", "PCYOX1", "PCYT1A", "PCYT2", "PDAP1", "PDCD2", "PDE1A", "PDE3A", "PDE5A", "PDE8B", "PDF", "PDGFA", "PDGFD", "PDHA1", "PDHB", "PDIA3", "PDIA4", "PDK1", "PDK2", "PDK3", "PDLIM1", "PDLIM3", "PDLIM5", "PDPN", "PDPR", "PDS5B", "PDXK", "PDZD8", "PEBP1", "PECAM1", "PELP1", "PEPD", "PER1", "PERP", "PEX16", "PEX2", "PEX6", "PFDN4", "PFKFB2", "PFKP", "PGAM5", "PGGHG", "PGGT1B", "PGK1", "PGP", "PGRMC1", "PGRMC2", "PGS1", "PHAX", "PHF20L1", "PHKB", "PHLDA3", "PIAS1", "PIAS2", "PIFO", "PIGBOS1", "PIGG", "PIGK", "PIGN", "PIGP", "PIGX", "PIK3CA", "PILRB", "PINK1", "PIP4K2A", "PISD", "PITPNA", "PJA2", "PKDCC", "PKIA", "PKIB", "PKNOX1", "PLA2G12A", "PLA2G4F", "PLA2R1", "PLAA", "PLCD3", "PLD1", "PLD5", "PLEKHA6", "PLEKHB1", "PLEKHF2", "PLEKHG1", "PLEKHG2", "PLEKHG3", "PLEKHO1", "PLEKHS1", "PLIN2", "PLK1", "PLK2", "PLOD2", "PLPP4", "PLRG1", "PLXDC1", "PLXDC2", "PLXND1", "PMPCA", "PNISR", "PNN", "PNO1", "PNPLA6", "PNPLA8", "PNPT1", "POGK", "POLDIP2", "POLI", "POLR1A", "POLR1B", "POLR1D", "POLR1E", "POLR2C", "POLR2D", "POLR2E", "POLR2J", "POLR2L", "POLR3B", "POLR3D", "POMT1", "PON1", "POP4", "POPDC3", "POT1", "POU6F1", "PPARD", "PPDPF", "PPDPFL", "PPFIBP2", "PPHLN1", "PPIF", "PPIL2", "PPIP5K2", "PPM1K", "PPP1CB", "PPP1R15A", "PPP1R1C", "PPP1R21", "PPP1R3G", "PPP1R7", "PPP2CA", "PPP2R1A", "PPP2R1B", "PPP2R2A", "PPP2R2C", "PPP2R5E", "PPP3CC", "PPP3R1", "PPP4R3A", "PPP6C", "PRCP", "PRDM15", "PRDM4", "PRDM5", "PRDX6", "PRELID2", "PRELID3B", "PRELP", "PREPL", "PREX1", "PRICKLE1", "PRKAG2", "PRKAR1A", "PRKD3", "PRMT2", "PROSER3", "PRPF38A", "PRPF38B", "PRPF4", "PRPSAP1", "PRR11", "PRR13", "PRSS12", "PRSS16", "PRTFDC1", "PSAP", "PSAPL1", "PSCA", "PSEN1", "PSMA2", "PSMA4", "PSMD13", "PSMD9", "PSME4", "PSMG4", "PSPH", "PSTPIP2", "PTBP2", "PTCD2", "PTCD3", "PTDSS1", "PTER", "PTGR2", "PTK2", "PTP4A2", "PTPN14", "PTPN21", "PTPRB", "PTTG1IP", "PUDP", "PUS7L", "PWP1", "PXDC1", "PXMP2", "PXMP4", "PYGL", "QDPR", "QKI", "QRSL1", "QSOX1", "RAB10", "RAB11A", "RAB18", "RAB21", "RAB22A", "RAB24", "RAB27A", "RAB2A", "RAB30", "RAB31", "RAB36", "RAB3B", "RAB3GAP1", "RAB3IP", "RAB40B", "RAB4A", "RAB5A", "RAB8A", "RABEP1", "RABGAP1", "RABGEF1", "RABIF", "RAD1", "RAD23B", "RAD51", "RAD52", "RAET1E", "RALA", "RAN", "RANBP1", "RANGAP1", "RAP1A", "RAPGEF3", "RARS2", "RASAL1", "RASGEF1C", "RASGRP3", "RBFA", "RBM12", "RBM18", "RBM23", "RBM28", "RBM3", "RBM38", "RBM7", "RBMS2", "RBX1", "RCAN3", "RCBTB1", "RCCD1", "RCHY1", "RCL1", "RCN1", "RDH11", "REEP5", "REG1A", "RELT", "REPS1", "REST", "RETREG2", "RETSAT", "REX1BD", "RFC5", "RFT1", "RFTN1", "RGS5", "RHOBTB2", "RHOH", "RHOJ", "RHPN1", "RIBC2", "RIC1", "RIF1", "RIIAD1", "RIMS3", "RIPOR2", "RITA1", "RMDN1", "RNASEK", "RNASEL", "RNASET2", "RNF111", "RNF121", "RNF14", "RNF141", "RNF166", "RNF167", "RNF207", "RNF24", "RNF41", "RNPEPL1", "ROBO4", "ROCK1", "ROCK1P1", "ROCK2", "RORA", "RP9P", "RPA1", "RPA2", "RPA3", "RPAIN", "RPAP2", "RPAP3", "RPF2", "RPGRIP1L", "RPL11", "RPL13", "RPL13A", "RPL14", "RPL22L1", "RPL23", "RPL23AP53", "RPL31", "RPL35A", "RPL37", "RPL37A", "RPL7L1", "RPN1", "RPP30", "RPP38", "RPRD2", "RPS10-NUDT3", "RPS14", "RPS19", "RPS20", "RPS23", "RPS27L", "RPS28", "RPS6", "RPUSD2", "RPUSD4", "RRAGA", "RRAGD", "RRAS2", "RRBP1", "RREB1", "RRM2B", "RRP1", "RRP15", "RRP7A", "RRP7BP", "RSBN1", "RSL1D1", "RSPH3", "RTEL1", "RTKN2", "RUBCNL", "RUFY3", "RWDD1", "RWDD2B", "RXRG", "RYK", "S100A1", "S100A14", "S100PBP", "SACM1L", "SAMD14", "SAP18", "SAR1A", "SAR1B", "SARAF", "SART1", "SART3", "SAYSD1", "SBF1", "SBF2", "SCAP", "SCARB1", "SCCPDH", "SCD5", "SCFD2", "SCML4", "SCO1", "SCP2", "SCRN2", "SCRN3", "SDC2", "SDHA", "SDHAF4", "SDHC", "SDK1", "SDR42E1", "SEC14L2", "SEC22A", "SEC23IP", "SEC62", "SEC63", "SELENOF", "SELENOI", "SELENOS", "SELENOT", "SELPLG", "SEMA3D", "SEMA4D", "SEMA6C", "SENP2", "SENP3", "SENP5", "SENP6", "SENP8", "SEPSECS", "SERINC3", "SERP1", "SERPINB1", "SERPINB9", "SERPINE1", "SERPINH1", "SERPINI2", "SESN3", "SETD4", "SETD6", "SETD9", "SETMAR", "SEZ6L2", "SF3B2", "SF3B3", "SFT2D2", "SFXN1", "SFXN2", "SGCA", "SGCB", "SGIP1", "SGO1", "SGPL1", "SGPP2", "SGSM3", "SH3BGRL", "SH3D21", "SH3GLB1", "SH3GLB2", "SH3TC2", "SHB", "SHC2", "SHC3", "SIGLEC16", "SIGLEC5", "SIN3B", "SIRT3", "SIX1", "SKA2", "SLAMF7", "SLC11A1", "SLC11A2", "SLC12A6", "SLC15A2", "SLC15A4", "SLC16A4", "SLC17A9", "SLC19A1", "SLC1A3", "SLC1A4", "SLC20A2", "SLC25A16", "SLC25A32", "SLC25A4", "SLC25A46", "SLC25A51", "SLC28A3", "SLC29A4", "SLC2A14", "SLC2A3", "SLC2A5", "SLC2A8", "SLC30A3", "SLC30A5", "SLC30A9", "SLC35A1", "SLC35A3", "SLC35B3", "SLC35B4", "SLC35C1", "SLC35C2", "SLC35E3", "SLC35F5", "SLC35G2", "SLC36A1", "SLC36A4", "SLC37A2", "SLC38A7", "SLC38A9", "SLC39A3", "SLC41A3", "SLC46A1", "SLC4A2", "SLC51A", "SLC5A12", "SLC6A11", "SLC6A8", "SLC7A5", "SLC9A3R2", "SLF1", "SLFN5", "SMAD2", "SMAD3", "SMAD5", "SMARCA4", "SMARCE1", "SMC2", "SMC3", "SMC4", "SMC6", "SMG9", "SMIM10L2A", "SMIM12", "SMIM14", "SMIM19", "SMIM24", "SMIM25", "SMIM30", "SMIM7", "SMIM8", "SMNDC1", "SMU1", "SMUG1", "SNAP29", "SNAPC1", "SNAPC3", "SNAPC5", "SNCA", "SNRNP25", "SNRNP27", "SNRNP35", "SNRNP48", "SNRPB2", "SNRPD3", "SNRPF", "SNU13", "SNX13", "SNX17", "SNX19", "SNX2", "SNX21", "SNX22", "SNX31", "SNX33", "SNX5", "SNX8", "SOAT1", "SOD2", "SORBS1", "SORBS2", "SP110", "SP3", "SPAG17", "SPARC", "SPART", "SPATA20", "SPATA33", "SPATS2L", "SPC24", "SPCS3", "SPDL1", "SPG21", "SPINK5", "SPINT1", "SPOCK2", "SPOCK3", "SPOPL", "SPPL2B", "SPPL3", "SPRR1A", "SPRYD4", "SPRYD7", "SPTLC1", "SPTLC2", "SPTLC3", "SQLE", "SRARP", "SRD5A3", "SREBF2", "SRGAP1", "SRI", "SRP14", "SRP19", "SRP72", "SRPRB", "SRSF1", "SRSF10", "SRSF3", "SRSF8", "SSBP2", "SSBP3", "SSC5D", "SSR1", "SSR3", "ST3GAL3", "ST3GAL6", "ST6GALNAC3", "ST7L", "STAG3L4", "STARD3NL", "STARD9", "STAT6", "STEAP4", "STK17A", "STK17B", "STK25", "STK33", "STK4", "STMN3", "STMP1", "STN1", "STOM", "STRAP", "STRIP2", "STT3A", "STX17", "STX2", "STX3", "STX6", "STX7", "STXBP5", "STXBP6", "STYK1", "SUDS3", "SUGT1", "SULF1", "SULT1A1", "SUMO3", "SUN2", "SVEP1", "SVIP", "SVOP", "SYCE1", "SYCE1L", "SYDE1", "SYN2", "SYNE2", "SYNE3", "SYNGR1", "SYNJ2BP-COX16", "SYT5", "SYTL4", "TACC2", "TADA2B", "TAF1C", "TAF8", "TANK", "TAPT1", "TARDBP", "TATDN2", "TATDN3", "TBC1D15", "TBC1D16", "TBC1D20", "TBC1D4", "TBL2", "TBL3" ] }' 'http://rest.ensembl.org/lookup/symbol/homo_sapiens'  -O -


In [198]:

json_str_4 = !wget -q --header='Content-type:application/json' --header='Accept:application/json' --post-data='{ "symbols" : ["TBX18", "TBX2", "TBXA2R", "TCEAL2", "TCEANC2", "TCF12", "TCFL5", "TCTEX1D1", "TDP2", "TDRD1", "TDRP", "TECPR2", "TELO2", "TES", "TESPA1", "TFB2M", "TFCP2", "TGFB2", "THAP5", "THBS2", "THEM4", "THEMIS2", "THOC5", "TIA1", "TIFA", "TIGAR", "TIMM10", "TIMM10B", "TIMM13", "TIMM21", "TIMMDC1", "TIPRL", "TK1", "TKT", "TLK1", "TLL1", "TLN2", "TLR3", "TLR4", "TM9SF3", "TMA16", "TMBIM1", "TMBIM4", "TMC8", "TMCO1", "TMCO3", "TMED2", "TMED4", "TMED5", "TMED7-TICAM2", "TMEM106B", "TMEM120B", "TMEM125", "TMEM126B", "TMEM127", "TMEM128", "TMEM135", "TMEM167A", "TMEM168", "TMEM176A", "TMEM183A", "TMEM191A", "TMEM192", "TMEM218", "TMEM220", "TMEM231", "TMEM242", "TMEM248", "TMEM255B", "TMEM268", "TMEM33", "TMEM38B", "TMEM45B", "TMEM50A", "TMEM53", "TMEM64", "TMEM70", "TMEM8B", "TMEM9", "TMEM98", "TMEM99", "TMOD3", "TMPRSS4", "TMTC1", "TMX1", "TMX4", "TNFAIP8", "TNFRSF11A", "TNIP1", "TNKS2", "TNNT1", "TNPO1", "TNS2", "TNS4", "TOLLIP", "TOMM22", "TOMM7", "TOMM70", "TOR1B", "TOX4", "TP53RK", "TPCN2", "TPM1", "TPP2", "TPST2", "TPT1", "TRABD", "TRAF1", "TRAF2", "TRAF3IP1", "TRAF6", "TRAK1", "TRAK2", "TRAM1", "TRAM2", "TRAPPC2L", "TRAPPC3", "TRAPPC5", "TREM1", "TRIM13", "TRIM16L", "TRIM24", "TRIM25", "TRIM38", "TRIM4", "TRIM5", "TRIM66", "TRIOBP", "TRIP11", "TRIP6", "TRMT10B", "TRMT6", "TRNT1", "TRPC4", "TRPM7", "TRUB2", "TSC22D1", "TSEN2", "TSPAN11", "TSPAN14", "TSPAN15", "TSPAN2", "TSPAN3", "TSPYL1", "TTC23", "TTC26", "TTC33", "TTC39B", "TTC5", "TTC9", "TTLL1", "TUBA1B", "TUBA3E", "TUSC2", "TUSC3", "TVP23C-CDRT4", "TWSG1", "TXN", "TXN2", "TXNDC15", "TXNDC17", "TXNL1", "TXNL4A", "TXNRD2", "TYRO3", "TYRP1", "TYSND1", "TYW1", "U2AF2", "U2SURP", "UBA5", "UBA52", "UBA6", "UBAP2", "UBE2D1", "UBE2D4", "UBE2E2", "UBE2G2", "UBE2J1", "UBE2J2", "UBE2L6", "UBE2R2", "UBE2W", "UBP1", "UBR1", "UBXN11", "UBXN6", "UCHL3", "UCK2", "UCP2", "UFSP2", "UGDH", "UGGT1", "ULBP1", "ULK4", "UMPS", "UNC13C", "UQCC2", "UQCR10", "UQCRQ", "URI1", "USHBP1", "USP14", "USP24", "USP3", "USP31", "USP35", "USP38", "USP4", "USP45", "USP48", "USP53", "USP6NL", "USP8", "USP9X", "USPL1", "UTP15", "UVSSA", "VAMP8", "VAPB", "VASH2", "VASP", "VCAN", "VEGFB", "VENTX", "VEZT", "VGLL3", "VHL", "VIM", "VKORC1L1", "VLDLR", "VMP1", "VNN1", "VPS13A", "VPS13D", "VPS26A", "VPS29", "VPS33A", "VPS37A", "VPS41", "VPS4A", "VPS50", "VRK3", "VSIG10L", "VSIR", "VSTM4", "VTA1", "VWA5A", "VWA5B1", "WARS2", "WASF2", "WASHC2A", "WBP11", "WBP4", "WDFY2", "WDPCP", "WDR26", "WDR3", "WDR35", "WDR36", "WDR4", "WDR41", "WDR48", "WDR55", "WDR59", "WDR60", "WDR7", "WDR73", "WDR90", "WDR92", "WDR93", "WFDC3", "WIPF1", "WIPI2", "WNT2B", "WNT5A", "WWC1", "WWP1", "XG", "XPA", "XPO7", "XPR1", "XRCC5", "XRRA1", "XYLT2", "YARS2", "YBEY", "YIPF4", "YIPF5", "YIPF6", "YKT6", "YME1L1", "YPEL1", "YTHDC2", "YWHAB", "ZADH2", "ZBTB25", "ZBTB44", "ZBTB8OS", "ZC2HC1A", "ZC3H7A", "ZC3HAV1", "ZDHHC11", "ZDHHC11B", "ZDHHC14", "ZDHHC2", "ZDHHC21", "ZDHHC24", "ZDHHC4", "ZDHHC8", "ZDHHC9", "ZFAND1", "ZFAND4", "ZFP28", "ZFYVE16", "ZFYVE21", "ZIK1", "ZMAT1", "ZMAT2", "ZMAT3", "ZMIZ2", "ZNF106", "ZNF134", "ZNF135", "ZNF174", "ZNF181", "ZNF200", "ZNF211", "ZNF24", "ZNF26", "ZNF264", "ZNF267", "ZNF268", "ZNF271P", "ZNF273", "ZNF276", "ZNF277", "ZNF316", "ZNF330", "ZNF34", "ZNF350", "ZNF37BP", "ZNF383", "ZNF385D", "ZNF398", "ZNF404", "ZNF410", "ZNF415", "ZNF417", "ZNF418", "ZNF419", "ZNF467", "ZNF471", "ZNF474", "ZNF480", "ZNF552", "ZNF558", "ZNF57", "ZNF577", "ZNF579", "ZNF587", "ZNF592", "ZNF593", "ZNF598", "ZNF641", "ZNF649", "ZNF697", "ZNF7", "ZNF701", "ZNF717", "ZNF737", "ZNF74", "ZNF766", "ZNF812P", "ZNF816-ZNF321P", "ZNF880", "ZNF891", "ZNHIT6", "ZNRD1", "ZRANB3", "ZSWIM7", "ZWILCH", "ZZEF1" ] }' 'http://rest.ensembl.org/lookup/symbol/homo_sapiens'  -O -


In [199]:
import json

ens_dict = json.loads(json_str[0])
ens_dict_2 = json.loads(json_str_2[0])
ens_dict_3 = json.loads(json_str_3[0])
ens_dict_4 = json.loads(json_str_4[0])


In [200]:

ens_ids = []
for _, row in merged_df_utr3.iterrows() :
    if row['gene'] in ens_dict :
        ens_ids.append(ens_dict[row['gene']]['id'])
    elif row['gene'] in ens_dict_2 :
        ens_ids.append(ens_dict_2[row['gene']]['id'])
    elif row['gene'] in ens_dict_3 :
        ens_ids.append(ens_dict_3[row['gene']]['id'])
    elif row['gene'] in ens_dict_4 :
        ens_ids.append(ens_dict_4[row['gene']]['id'])
    else :
        ens_ids.append("Missing")

merged_df_utr3['ens_id'] = ens_ids


In [29]:
import os
import subprocess

p_values = []
effect_sizes = []
effect_size_ses = []

i = 0
for _, row in merged_df_intron.iterrows() :
    i
    if i % 10 == 0 :
        print("Processing SNP " + str(i) + "...")
    
    snp_id = row['snp_id'][:-4]
    ens_id = row['ens_id']
    tissue_type = row['tissue_type']
    
    url = 'https://www.ebi.ac.uk/eqtl/api/associations/' + snp_id + '?study=GTEx_V8&size=100&qtl_group=' + tissue_type + '&gene_id=' + ens_id
    
    cmd = ['wget', '-q', str(url), '-O', '-']

    json_str = str(subprocess.run(cmd, stdout=subprocess.PIPE).stdout)
    
    if len(json_str) >= 20 :
        try :
            json_dict = json.loads(json_str[2:-1])
            
            if '_embedded' in json_dict and 'associations' in json_dict['_embedded'] and len(json_dict['_embedded']['associations']) > 0 and '0' in json_dict['_embedded']['associations'] :
                p_values.append(json_dict['_embedded']['associations']['0']['pvalue'])
                effect_sizes.append(json_dict['_embedded']['associations']['0']['beta'])
                effect_size_ses.append(json_dict['_embedded']['associations']['0']['se'])
            else :
                p_values.append(-1)
                effect_sizes.append(-10000.)
                effect_size_ses.append(-10000.)
                continue
            
        except Exception as e :
            p_values.append(-1)
            effect_sizes.append(-10000.)
            effect_size_ses.append(-10000.)
    else :
        p_values.append(-1)
        effect_sizes.append(-10000.)
        effect_size_ses.append(-10000.)
    
    i += 1
    

merged_df_intron['eqtl_p_val'] = p_values
merged_df_intron['eqtl_effect_size'] = effect_sizes
merged_df_intron['eqtl_effect_size_se'] = effect_size_ses


Processing SNP 0...
Processing SNP 10...
Processing SNP 20...
Processing SNP 30...
Processing SNP 40...
Processing SNP 50...
Processing SNP 60...
Processing SNP 70...
Processing SNP 80...
Processing SNP 90...
Processing SNP 100...
Processing SNP 110...
Processing SNP 120...
Processing SNP 130...
Processing SNP 140...
Processing SNP 150...
Processing SNP 160...
Processing SNP 170...
Processing SNP 180...
Processing SNP 190...
Processing SNP 200...
Processing SNP 210...
Processing SNP 220...
Processing SNP 230...
Processing SNP 240...
Processing SNP 250...
Processing SNP 260...
Processing SNP 270...
Processing SNP 280...
Processing SNP 290...
Processing SNP 300...
Processing SNP 310...
Processing SNP 320...
Processing SNP 330...
Processing SNP 340...
Processing SNP 350...
Processing SNP 360...
Processing SNP 370...
Processing SNP 380...
Processing SNP 390...
Processing SNP 400...
Processing SNP 410...
Processing SNP 420...
Processing SNP 430...
Processing SNP 440...
Processing SNP 450...

Processing SNP 3580...
Processing SNP 3590...
Processing SNP 3600...
Processing SNP 3610...
Processing SNP 3620...
Processing SNP 3630...
Processing SNP 3640...
Processing SNP 3650...
Processing SNP 3660...
Processing SNP 3670...
Processing SNP 3680...
Processing SNP 3690...
Processing SNP 3700...
Processing SNP 3710...
Processing SNP 3720...
Processing SNP 3730...
Processing SNP 3740...
Processing SNP 3750...
Processing SNP 3760...
Processing SNP 3770...
Processing SNP 3780...
Processing SNP 3790...
Processing SNP 3800...
Processing SNP 3810...
Processing SNP 3820...
Processing SNP 3830...
Processing SNP 3840...
Processing SNP 3850...
Processing SNP 3860...
Processing SNP 3870...
Processing SNP 3880...
Processing SNP 3890...
Processing SNP 3900...
Processing SNP 3910...
Processing SNP 3920...
Processing SNP 3930...
Processing SNP 3940...
Processing SNP 3950...
Processing SNP 3960...
Processing SNP 3970...
Processing SNP 3980...
Processing SNP 3990...
Processing SNP 4000...
Processing 

Processing SNP 7120...
Processing SNP 7130...
Processing SNP 7140...
Processing SNP 7150...
Processing SNP 7160...
Processing SNP 7170...
Processing SNP 7180...
Processing SNP 7190...
Processing SNP 7200...
Processing SNP 7210...
Processing SNP 7220...
Processing SNP 7230...
Processing SNP 7240...
Processing SNP 7250...
Processing SNP 7260...
Processing SNP 7270...
Processing SNP 7280...
Processing SNP 7290...
Processing SNP 7300...
Processing SNP 7310...
Processing SNP 7320...
Processing SNP 7320...
Processing SNP 7330...
Processing SNP 7340...
Processing SNP 7350...
Processing SNP 7360...
Processing SNP 7370...
Processing SNP 7380...
Processing SNP 7390...
Processing SNP 7400...
Processing SNP 7410...
Processing SNP 7420...
Processing SNP 7430...
Processing SNP 7440...
Processing SNP 7450...
Processing SNP 7460...
Processing SNP 7470...
Processing SNP 7480...
Processing SNP 7490...
Processing SNP 7500...
Processing SNP 7510...
Processing SNP 7520...
Processing SNP 7530...
Processing 

Processing SNP 10600...
Processing SNP 10610...
Processing SNP 10620...
Processing SNP 10630...
Processing SNP 10640...
Processing SNP 10650...
Processing SNP 10660...
Processing SNP 10670...
Processing SNP 10680...
Processing SNP 10690...
Processing SNP 10700...
Processing SNP 10710...
Processing SNP 10720...
Processing SNP 10730...
Processing SNP 10740...
Processing SNP 10750...
Processing SNP 10760...
Processing SNP 10770...
Processing SNP 10780...
Processing SNP 10790...
Processing SNP 10800...
Processing SNP 10810...
Processing SNP 10820...
Processing SNP 10830...
Processing SNP 10840...
Processing SNP 10850...
Processing SNP 10860...
Processing SNP 10870...
Processing SNP 10880...
Processing SNP 10890...
Processing SNP 10900...
Processing SNP 10910...
Processing SNP 10920...
Processing SNP 10930...
Processing SNP 10940...
Processing SNP 10950...
Processing SNP 10960...
Processing SNP 10970...
Processing SNP 10980...
Processing SNP 10990...
Processing SNP 11000...
Processing SNP 1

Processing SNP 14010...
Processing SNP 14020...
Processing SNP 14030...
Processing SNP 14040...
Processing SNP 14050...
Processing SNP 14060...
Processing SNP 14070...
Processing SNP 14080...
Processing SNP 14090...
Processing SNP 14100...
Processing SNP 14110...
Processing SNP 14120...
Processing SNP 14130...
Processing SNP 14140...
Processing SNP 14150...
Processing SNP 14160...
Processing SNP 14170...
Processing SNP 14180...
Processing SNP 14190...
Processing SNP 14200...
Processing SNP 14210...
Processing SNP 14220...
Processing SNP 14230...
Processing SNP 14240...
Processing SNP 14250...
Processing SNP 14260...
Processing SNP 14270...
Processing SNP 14280...
Processing SNP 14290...
Processing SNP 14300...
Processing SNP 14310...
Processing SNP 14320...
Processing SNP 14330...
Processing SNP 14340...
Processing SNP 14350...
Processing SNP 14360...
Processing SNP 14370...
Processing SNP 14380...
Processing SNP 14390...
Processing SNP 14400...
Processing SNP 14410...
Processing SNP 1

Processing SNP 17390...
Processing SNP 17400...
Processing SNP 17410...
Processing SNP 17420...
Processing SNP 17430...
Processing SNP 17440...
Processing SNP 17450...
Processing SNP 17460...
Processing SNP 17470...
Processing SNP 17480...
Processing SNP 17490...
Processing SNP 17500...
Processing SNP 17510...
Processing SNP 17520...
Processing SNP 17530...
Processing SNP 17540...
Processing SNP 17550...
Processing SNP 17560...
Processing SNP 17570...
Processing SNP 17580...
Processing SNP 17590...
Processing SNP 17600...
Processing SNP 17610...
Processing SNP 17620...
Processing SNP 17630...
Processing SNP 17640...
Processing SNP 17650...
Processing SNP 17660...
Processing SNP 17670...
Processing SNP 17680...
Processing SNP 17690...
Processing SNP 17700...
Processing SNP 17710...
Processing SNP 17720...
Processing SNP 17730...
Processing SNP 17740...
Processing SNP 17750...
Processing SNP 17760...
Processing SNP 17770...
Processing SNP 17780...
Processing SNP 17790...
Processing SNP 1

In [205]:
import os
import subprocess

p_values = []
effect_sizes = []
effect_size_ses = []

eqtl_dict = {}

i = 0
for _, row in merged_df_utr3.iterrows() :
    i
    if i % 10 == 0 :
        print("Processing SNP " + str(i) + "...")
    
    snp_id = row['snp_id'][:-4]
    ens_id = row['ens_id']
    tissue_type = row['tissue_type']
    
    if snp_id in eqtl_dict :
        if tissue_type in eqtl_dict[snp_id] :
            p_values.append(eqtl_dict[snp_id][tissue_type]['p_value'])
            effect_sizes.append(eqtl_dict[snp_id][tissue_type]['effect_size'])
            effect_size_ses.append(eqtl_dict[snp_id][tissue_type]['effect_size_se'])
        else :
            p_values.append(-1)
            effect_sizes.append(-10000.)
            effect_size_ses.append(-10000.)
        
        i += 1
        continue
    
    url = 'https://www.ebi.ac.uk/eqtl/api/associations/' + snp_id + '?study=GTEx_V8&size=100&gene_id=' + ens_id
    
    cmd = ['wget', '-q', str(url), '-O', '-']

    json_str = str(subprocess.run(cmd, stdout=subprocess.PIPE).stdout)
    
    if len(json_str) >= 20 :
        try :
            json_dict = json.loads(json_str[2:-1])
            
            if '_embedded' in json_dict and 'associations' in json_dict['_embedded'] :
                eqtl_dict[snp_id] = {}
                for k in range(len(json_dict['_embedded']['associations'])) :
                    eqtl_dict[snp_id][json_dict['_embedded']['associations'][str(k)]['qtl_group']] = {
                        'p_value' : json_dict['_embedded']['associations'][str(k)]['pvalue'],
                        'effect_size' : json_dict['_embedded']['associations'][str(k)]['beta'],
                        'effect_size_se' : json_dict['_embedded']['associations'][str(k)]['se']
                    }
                
                if tissue_type in eqtl_dict[snp_id] :
                    p_values.append(eqtl_dict[snp_id][tissue_type]['p_value'])
                    effect_sizes.append(eqtl_dict[snp_id][tissue_type]['effect_size'])
                    effect_size_ses.append(eqtl_dict[snp_id][tissue_type]['effect_size_se'])
                else :
                    p_values.append(-1)
                    effect_sizes.append(-10000.)
                    effect_size_ses.append(-10000.)
                    i += 1
                    continue
            else :
                p_values.append(-1)
                effect_sizes.append(-10000.)
                effect_size_ses.append(-10000.)
                i += 1
                continue
            
        except Exception as e :
            p_values.append(-1)
            effect_sizes.append(-10000.)
            effect_size_ses.append(-10000.)
    else :
        p_values.append(-1)
        effect_sizes.append(-10000.)
        effect_size_ses.append(-10000.)
    
    i += 1
    

merged_df_utr3['eqtl_p_val'] = p_values
merged_df_utr3['eqtl_effect_size'] = effect_sizes
merged_df_utr3['eqtl_effect_size_se'] = effect_size_ses


Processing SNP 0...
Processing SNP 10...
Processing SNP 20...
Processing SNP 30...
Processing SNP 40...
Processing SNP 50...
Processing SNP 60...
Processing SNP 70...
Processing SNP 80...
Processing SNP 90...
Processing SNP 100...
Processing SNP 110...
Processing SNP 120...
Processing SNP 130...
Processing SNP 140...
Processing SNP 150...
Processing SNP 160...
Processing SNP 170...
Processing SNP 180...
Processing SNP 190...
Processing SNP 200...
Processing SNP 210...
Processing SNP 220...
Processing SNP 230...
Processing SNP 240...
Processing SNP 250...
Processing SNP 260...
Processing SNP 270...
Processing SNP 280...
Processing SNP 290...
Processing SNP 300...
Processing SNP 310...
Processing SNP 320...
Processing SNP 330...
Processing SNP 340...
Processing SNP 350...
Processing SNP 360...
Processing SNP 370...
Processing SNP 380...
Processing SNP 390...
Processing SNP 400...
Processing SNP 410...
Processing SNP 420...
Processing SNP 430...
Processing SNP 440...
Processing SNP 450...

Processing SNP 3630...
Processing SNP 3640...
Processing SNP 3650...
Processing SNP 3660...
Processing SNP 3670...
Processing SNP 3680...
Processing SNP 3690...
Processing SNP 3700...
Processing SNP 3710...
Processing SNP 3720...
Processing SNP 3730...
Processing SNP 3740...
Processing SNP 3750...
Processing SNP 3760...
Processing SNP 3770...
Processing SNP 3780...
Processing SNP 3790...
Processing SNP 3800...
Processing SNP 3810...
Processing SNP 3820...
Processing SNP 3830...
Processing SNP 3840...
Processing SNP 3850...
Processing SNP 3860...
Processing SNP 3870...
Processing SNP 3880...
Processing SNP 3890...
Processing SNP 3900...
Processing SNP 3910...
Processing SNP 3920...
Processing SNP 3930...
Processing SNP 3940...
Processing SNP 3950...
Processing SNP 3960...
Processing SNP 3970...
Processing SNP 3980...
Processing SNP 3990...
Processing SNP 4000...
Processing SNP 4010...
Processing SNP 4020...
Processing SNP 4030...
Processing SNP 4040...
Processing SNP 4050...
Processing 

Processing SNP 7200...
Processing SNP 7210...
Processing SNP 7220...
Processing SNP 7230...
Processing SNP 7240...
Processing SNP 7250...
Processing SNP 7260...
Processing SNP 7270...
Processing SNP 7280...
Processing SNP 7290...
Processing SNP 7300...
Processing SNP 7310...
Processing SNP 7320...
Processing SNP 7330...
Processing SNP 7340...
Processing SNP 7350...
Processing SNP 7360...
Processing SNP 7370...
Processing SNP 7380...
Processing SNP 7390...
Processing SNP 7400...
Processing SNP 7410...
Processing SNP 7420...
Processing SNP 7430...
Processing SNP 7440...
Processing SNP 7450...
Processing SNP 7460...
Processing SNP 7470...
Processing SNP 7480...
Processing SNP 7490...
Processing SNP 7500...
Processing SNP 7510...
Processing SNP 7520...
Processing SNP 7530...
Processing SNP 7540...
Processing SNP 7550...
Processing SNP 7560...
Processing SNP 7570...
Processing SNP 7580...
Processing SNP 7590...
Processing SNP 7600...
Processing SNP 7610...
Processing SNP 7620...
Processing 

Processing SNP 10750...
Processing SNP 10760...
Processing SNP 10770...
Processing SNP 10780...
Processing SNP 10790...
Processing SNP 10800...
Processing SNP 10810...
Processing SNP 10820...
Processing SNP 10830...
Processing SNP 10840...
Processing SNP 10850...
Processing SNP 10860...
Processing SNP 10870...
Processing SNP 10880...
Processing SNP 10890...
Processing SNP 10900...
Processing SNP 10910...
Processing SNP 10920...
Processing SNP 10930...
Processing SNP 10940...
Processing SNP 10950...
Processing SNP 10960...
Processing SNP 10970...
Processing SNP 10980...
Processing SNP 10990...
Processing SNP 11000...
Processing SNP 11010...
Processing SNP 11020...
Processing SNP 11030...
Processing SNP 11040...
Processing SNP 11050...
Processing SNP 11060...
Processing SNP 11070...
Processing SNP 11080...
Processing SNP 11090...
Processing SNP 11100...
Processing SNP 11110...
Processing SNP 11120...
Processing SNP 11130...
Processing SNP 11140...
Processing SNP 11150...
Processing SNP 1

Processing SNP 14230...
Processing SNP 14240...
Processing SNP 14250...
Processing SNP 14260...
Processing SNP 14270...
Processing SNP 14280...
Processing SNP 14290...
Processing SNP 14300...
Processing SNP 14310...
Processing SNP 14320...
Processing SNP 14330...
Processing SNP 14340...
Processing SNP 14350...
Processing SNP 14360...
Processing SNP 14370...
Processing SNP 14380...
Processing SNP 14390...
Processing SNP 14400...
Processing SNP 14410...
Processing SNP 14420...
Processing SNP 14430...
Processing SNP 14440...
Processing SNP 14450...
Processing SNP 14460...
Processing SNP 14470...
Processing SNP 14480...
Processing SNP 14490...
Processing SNP 14500...
Processing SNP 14510...
Processing SNP 14520...
Processing SNP 14530...
Processing SNP 14540...
Processing SNP 14550...
Processing SNP 14560...
Processing SNP 14570...
Processing SNP 14580...
Processing SNP 14590...
Processing SNP 14600...
Processing SNP 14610...
Processing SNP 14620...
Processing SNP 14630...
Processing SNP 1

...

Processing SNP 48760...
Processing SNP 48770...
Processing SNP 48780...
Processing SNP 48790...
Processing SNP 48800...
Processing SNP 48810...
Processing SNP 48820...
Processing SNP 48830...
Processing SNP 48840...
Processing SNP 48850...
Processing SNP 48860...
Processing SNP 48870...
Processing SNP 48880...
Processing SNP 48890...
Processing SNP 48900...
Processing SNP 48910...
Processing SNP 48920...
Processing SNP 48930...
Processing SNP 48940...
Processing SNP 48950...
Processing SNP 48960...
Processing SNP 48970...
Processing SNP 48980...
Processing SNP 48990...
Processing SNP 49000...
Processing SNP 49010...
Processing SNP 49020...
Processing SNP 49030...
Processing SNP 49040...
Processing SNP 49050...
Processing SNP 49060...
Processing SNP 49070...
Processing SNP 49080...
Processing SNP 49090...
Processing SNP 49100...
Processing SNP 49110...
Processing SNP 49120...
Processing SNP 49130...
Processing SNP 49140...
Processing SNP 49150...
Processing SNP 49160...
Processing SNP 4

Processing SNP 52180...
Processing SNP 52190...
Processing SNP 52200...
Processing SNP 52210...
Processing SNP 52220...
Processing SNP 52230...
Processing SNP 52240...
Processing SNP 52250...
Processing SNP 52260...
Processing SNP 52270...
Processing SNP 52280...
Processing SNP 52290...
Processing SNP 52300...
Processing SNP 52310...
Processing SNP 52320...
Processing SNP 52330...
Processing SNP 52340...
Processing SNP 52350...
Processing SNP 52360...
Processing SNP 52370...
Processing SNP 52380...
Processing SNP 52390...
Processing SNP 52400...
Processing SNP 52410...
Processing SNP 52420...
Processing SNP 52430...
Processing SNP 52440...
Processing SNP 52450...
Processing SNP 52460...
Processing SNP 52470...
Processing SNP 52480...
Processing SNP 52490...
Processing SNP 52500...
Processing SNP 52510...
Processing SNP 52520...
Processing SNP 52530...
Processing SNP 52540...
Processing SNP 52550...
Processing SNP 52560...
Processing SNP 52570...
Processing SNP 52580...
Processing SNP 5

Processing SNP 55810...
Processing SNP 55820...
Processing SNP 55830...
Processing SNP 55840...
Processing SNP 55850...
Processing SNP 55860...
Processing SNP 55870...
Processing SNP 55880...
Processing SNP 55890...
Processing SNP 55900...
Processing SNP 55910...
Processing SNP 55920...
Processing SNP 55930...
Processing SNP 55940...
Processing SNP 55950...
Processing SNP 55960...
Processing SNP 55970...
Processing SNP 55980...
Processing SNP 55990...
Processing SNP 56000...
Processing SNP 56010...
Processing SNP 56020...
Processing SNP 56030...
Processing SNP 56040...
Processing SNP 56050...
Processing SNP 56060...
Processing SNP 56070...
Processing SNP 56080...
Processing SNP 56090...
Processing SNP 56100...
Processing SNP 56110...
Processing SNP 56120...
Processing SNP 56130...
Processing SNP 56140...
Processing SNP 56150...
Processing SNP 56160...
Processing SNP 56170...
Processing SNP 56180...
Processing SNP 56190...
Processing SNP 56200...
Processing SNP 56210...
Processing SNP 5

Processing SNP 59250...
Processing SNP 59260...
Processing SNP 59270...
Processing SNP 59280...
Processing SNP 59290...
Processing SNP 59300...
Processing SNP 59310...
Processing SNP 59320...
Processing SNP 59330...
Processing SNP 59340...
Processing SNP 59350...
Processing SNP 59360...
Processing SNP 59370...
Processing SNP 59380...
Processing SNP 59390...
Processing SNP 59400...
Processing SNP 59410...
Processing SNP 59420...
Processing SNP 59430...
Processing SNP 59440...
Processing SNP 59450...
Processing SNP 59460...
Processing SNP 59470...
Processing SNP 59480...
Processing SNP 59490...
Processing SNP 59500...
Processing SNP 59510...
Processing SNP 59520...
Processing SNP 59530...
Processing SNP 59540...
Processing SNP 59550...
Processing SNP 59560...
Processing SNP 59570...
Processing SNP 59580...
Processing SNP 59590...
Processing SNP 59600...
Processing SNP 59610...
Processing SNP 59620...
Processing SNP 59630...
Processing SNP 59640...
Processing SNP 59650...
Processing SNP 5

Processing SNP 62670...
Processing SNP 62680...
Processing SNP 62690...
Processing SNP 62700...
Processing SNP 62710...
Processing SNP 62720...
Processing SNP 62730...
Processing SNP 62740...
Processing SNP 62750...
Processing SNP 62760...
Processing SNP 62770...
Processing SNP 62780...
Processing SNP 62790...
Processing SNP 62800...
Processing SNP 62810...
Processing SNP 62820...
Processing SNP 62830...
Processing SNP 62840...
Processing SNP 62850...
Processing SNP 62860...
Processing SNP 62870...
Processing SNP 62880...
Processing SNP 62890...
Processing SNP 62900...
Processing SNP 62910...
Processing SNP 62920...
Processing SNP 62930...
Processing SNP 62940...
Processing SNP 62950...
Processing SNP 62960...
Processing SNP 62970...
Processing SNP 62980...
Processing SNP 62990...
Processing SNP 63000...
Processing SNP 63010...
Processing SNP 63020...
Processing SNP 63030...
Processing SNP 63040...
Processing SNP 63050...
Processing SNP 63060...
Processing SNP 63070...
Processing SNP 6

In [33]:

merged_df_intron.to_csv("polyadb_merged_intron_eqtl" + version_suffix + "_SNPs.csv", sep='\t')


In [208]:

merged_df_utr3.to_csv("polyadb_merged_utr3_eqtl" + version_suffix + "_SNPs.csv", sep='\t')


In [8]:
#Print intron statistics

merged_df_intron = pd.read_csv("polyadb_merged_intron_eqtl" + version_suffix + "_SNPs.csv", sep='\t')
print(len(merged_df_intron['snp_id'].unique()))

merged_df_intron = merged_df_intron.query("eqtl_effect_size > -10000. and eqtl_p_val < 1e-5")
print(len(merged_df_intron['snp_id'].unique()))

#Print utr3 statistics

merged_df_utr3 = pd.read_csv("polyadb_merged_utr3_eqtl" + version_suffix + "_SNPs.csv", sep='\t')
print(len(merged_df_utr3['snp_id'].unique()))

merged_df_utr3 = merged_df_utr3.query("eqtl_effect_size > -10000. and eqtl_p_val < 1e-5")
print(len(merged_df_utr3['snp_id'].unique()))


2100
1007
4706
2225
